## SRJ - FeS

In [56]:
## Usage of basis set
import basis_set_exchange as bse
print(bse.get_all_basis_names())

basis = bse.get_basis('ANO-RCC-MB', fmt='nwchem')
#print(basis)

import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize

#qml.data.load("qchem", molname = "FeS", basis = basis)

n_basis = qml.qchem.mol_basis_data('ANO-RCC-MB', ['Fe', 'S'], load_data=basis)
print(n_basis)


['2ZaP', '2ZaPa-NR', '2ZaPa-NR-CV', '3-21G', '3ZaP', '3ZaPa-NR', '3ZaPa-NR-CV', '4-31G', '4ZaP', '4ZaPa-NR', '4ZaPa-NR-CV', '5-21G', '5ZaP', '5ZaPa-NR', '5ZaPa-NR-CV', '6-21G', '6-31++G', '6-31++G*', '6-31++G**', '6-31++G**-J', '6-31+G', '6-31+G*', '6-31+G**', '6-31+G*-J', '6-311++G', '6-311++G(2d,2p)', '6-311++G(3df,3pd)', '6-311++G*', '6-311++G**', '6-311++G**-J', '6-311+G', '6-311+G(2d,p)', '6-311+G*', '6-311+G**', '6-311+G*-J', '6-311G', '6-311G(2df,2pd)', '6-311G(d,p)', '6-311G*', '6-311G**', '6-311G**-RIFIT', '6-311G-J', '6-311xxG(d,p)', '6-31G', '6-31G(2df,p)', '6-31G(3df,3pd)', '6-31G(d,p)', '6-31G*', '6-31G**', '6-31G**-RIFIT', '6-31G*-Blaudeau', '6-31G-Blaudeau', '6-31G-J', '6ZaP', '6ZaPa-NR', '7ZaPa-NR', 'AHGBS-5', 'AHGBS-7', 'AHGBS-9', 'AHGBSP1-5', 'AHGBSP1-7', 'AHGBSP1-9', 'AHGBSP2-5', 'AHGBSP2-7', 'AHGBSP2-9', 'AHGBSP3-5', 'AHGBSP3-7', 'AHGBSP3-9', 'ANO-DK3', 'ANO-R', 'ANO-R0', 'ANO-R1', 'ANO-R2', 'ANO-R3', 'ANO-RCC', 'ANO-RCC-MB', 'ANO-RCC-VDZ', 'ANO-RCC-VDZP', 'ANO-RCC-

In [10]:
n_basis

([19, 9],
 (((0, 0, 0),
   [4316265.0,
    646342.4,
    147089.7,
    41661.52,
    13590.77,
    4905.75,
    1912.746,
    792.6043,
    344.8065,
    155.8999,
    72.23091,
    32.72506,
    15.66762,
    7.503483,
    3.312223,
    1.558471,
    0.683914,
    0.146757,
    0.070583,
    0.031449,
    0.01258],
   [0.00015003,
    0.00043597,
    0.00120365,
    0.00312635,
    0.00814591,
    0.02133892,
    0.05470838,
    0.12845394,
    0.25203824,
    0.35484986,
    0.27043078,
    0.06476086,
    -0.00110466,
    0.00184555,
    -0.000856,
    0.00037119,
    -0.00014687,
    6.097e-05,
    -5.789e-05,
    2.77e-05,
    -7.22e-06]),
  ((0, 0, 0),
   [4316265.0,
    646342.4,
    147089.7,
    41661.52,
    13590.77,
    4905.75,
    1912.746,
    792.6043,
    344.8065,
    155.8999,
    72.23091,
    32.72506,
    15.66762,
    7.503483,
    3.312223,
    1.558471,
    0.683914,
    0.146757,
    0.070583,
    0.031449,
    0.01258],
   [-4.622e-05,
    -0.00013445,
    -0

In [13]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
theta = np.pi/4
operator_check = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 1.826
symbols = ["Fe","S"]
electrons = 4
orbitals = 12
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 0*r_bohr], [0.0, 0.0, 1*r_bohr]])
molecule = qml.qchem.Molecule(symbols, coordinates, basis_name='ano-rcc-mb')


H, qubits = qml.qchem.molecular_hamiltonian(molecule)
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[3840] = 1.0 # Every molecule change, you need to change this index #3840
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the ash_excitationis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    #For generating the new state, they are adding the excitation in the exponent of the rotation, thats 
    #why we need to use qml.FermionicSE or qml.FermionicDE
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=theta, wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=theta, wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
     
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

            #print('D-Generating functions are', super2)
    #print('Total len of Generating functions is', len(generatingfns))
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    print('The length of operator pool', len(operator_pool))


    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
        
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already checked
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    #print('Print, if this is activated - HF state')
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
        else:
            print('Skipping individual excitation state for j < 2')
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H)
        print('Ham matrix is', Ham_matrix)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                print('The value of i and j is', i, j)
                S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy)    
    return ash_excitation, states,eig,gs_energy, Ham_matrix


ValueError: Currently, the only supported basis sets are 'sto-3g', '6-31g', '6-311g' and 'cc-pvdz'. Please consider using `load_data=True` to download the basis set from an external library that can be installed with: pip install basis-set-exchange.

In [55]:
import pyscf
import basis_set_exchange as bse
#print(bse.get_all_basis_names())

basis = bse.get_basis('ANO-RCC-MB', elements = ['Fe', 'S'], fmt = 'gaussian')
print(basis)
#print(dict(basis))
#alpha = basis['exponents']
#print(alpha)

RuntimeError: Unknown basis set format "gaussian"

In [49]:
basis

{'molssi_bse_schema': {'schema_type': 'complete', 'schema_version': '0.1'},
 'revision_description': 'Data from OpenMolCAS',
 'revision_date': '2019-03-05',
 'elements': {'16': {'electron_shells': [{'function_type': 'gto',
     'region': '',
     'angular_momentum': [0],
     'exponents': ['346348.23',
      '49391.146',
      '14610.990',
      '5187.2095',
      '1980.9676',
      '784.63139',
      '317.32779',
      '130.01976',
      '53.738208',
      '22.345896',
      '9.3332512',
      '3.9111868',
      '1.6432066',
      '.69174560',
      '.29167360',
      '.12314410',
      '.04925760'],
     'coefficients': [['0.00029092',
       '0.00094665',
       '0.00211923',
       '0.00602161',
       '0.01659726',
       '0.04683989',
       '0.12242581',
       '0.27022435',
       '0.40541657',
       '0.25571527',
       '0.02671693',
       '0.00390332',
       '-0.00204189',
       '0.00149034',
       '-0.00062107',
       '0.00044167',
       '-0.00000324'],
      ['-0.000

In [53]:
alpha = np.array([[],
                [3.42525091, 0.62391373, 0.1688554]], requires_grad = True) #Exponents

KeyError: 'exponent'

In [40]:
import pennylane as qml
from pennylane import numpy as np

# Define molecule
symbols = ['Fe', 'S']
geometry = np.array([[0.0, 0.0, 0.0],
                     [0.0, 0.0, 2.0]])

basis_name = "ANO-RCC-MB"  # The name must match the BSE convention

# Load basis set data for the molecule
n_basis, basis_data = qml.qchem.mol_basis_data(basis_name, symbols, load_data=True)
#print(n_basis)
#print(basis_data)

alpha = basis_data
print(alpha)
#coeff = basis_data['coefficients']

#class Molecule(symbols, coordinates, charge=0, mult=1, basis_name='sto-3g', name='molecule', load_data=False, l=None, alpha=None, coeff=None, normalize=True, unit='bohr')
#H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, method="pyscf", alpha=alpha, coeff=coeff)
molecule = qml.qchem.Molecule(symbols, geometry, load_data = True, alpha = basis_data, coeff = basis_data['coefficients'])


H, qubits = qml.qchem.molecular_hamiltonian(molecule)
print(H)




(((0, 0, 0), [4316265.0, 646342.4, 147089.7, 41661.52, 13590.77, 4905.75, 1912.746, 792.6043, 344.8065, 155.8999, 72.23091, 32.72506, 15.66762, 7.503483, 3.312223, 1.558471, 0.683914, 0.146757, 0.070583, 0.031449, 0.01258], [0.00015003, 0.00043597, 0.00120365, 0.00312635, 0.00814591, 0.02133892, 0.05470838, 0.12845394, 0.25203824, 0.35484986, 0.27043078, 0.06476086, -0.00110466, 0.00184555, -0.000856, 0.00037119, -0.00014687, 6.097e-05, -5.789e-05, 2.77e-05, -7.22e-06]), ((0, 0, 0), [4316265.0, 646342.4, 147089.7, 41661.52, 13590.77, 4905.75, 1912.746, 792.6043, 344.8065, 155.8999, 72.23091, 32.72506, 15.66762, 7.503483, 3.312223, 1.558471, 0.683914, 0.146757, 0.070583, 0.031449, 0.01258], [-4.622e-05, -0.00013445, -0.00037184, -0.00096889, -0.00253948, -0.00673001, -0.0176816, -0.0437541, -0.09601111, -0.16998599, -0.18456376, 0.058263, 0.52163758, 0.49331062, 0.0863267, -0.00285017, 0.00165569, -0.00049176, 0.00047608, -0.0002282, 6.297e-05]), ((0, 0, 0), [4316265.0, 646342.4, 147089

TypeError: tuple indices must be integers or slices, not str

In [15]:
import pennylane as qml

n_basis, params = qml.qchem.mol_basis_data('sto-3g', ['H', 'H'])
print(n_basis)
print(params)

[1, 1]
(((0, 0, 0), [3.425250914, 0.6239137298, 0.168855404], [0.1543289673, 0.5353281423, 0.4446345422]), ((0, 0, 0), [3.425250914, 0.6239137298, 0.168855404], [0.1543289673, 0.5353281423, 0.4446345422]))


## SRJ - Present 

In [ ]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.RHF(mol_pyscf)
rhf.kernel()

one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc')
two_ao = mol_pyscf.intor('int2e_sph')

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)
core_constant = np.array([rhf.energy_nuc()])

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)
#print(H)


## SRJ - Hamiltonian generation successful

In [1]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')
#basis_dict = bse.get_basis('ANO-RCC-MB', fmt='nwchem')

## Working code

In [11]:
import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged)
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.UHF(mol_pyscf)
rhf.kernel()


print("Atom list:", mol_pyscf.atom)
#print("Basis used:", mol_pyscf.basis)
print("Charge:", mol_pyscf.charge)
print("Spin:", mol_pyscf.spin)
print("Number of electrons:", mol_pyscf.nelectron)

# === ACTIVE SPACE IMPLEMENTATION ===
# Step 1: Define active space parameters
norb_active = 6      # Number of active orbitals
nelec_active = 6     # Number of active electrons

# Step 2: Set up CASSCF to get active space integrals
mycas = mcscf.CASSCF(rhf, norb_active, nelec_active)
mycas.kernel()  # Run CASSCF to get the ground state energy and active space integrals
print("Hartree-Fock Ground State Energy:", rhf.e_tot)
print("CASSCF Ground State Energy:", mycas.e_tot)

# Step 3: Get effective active space integrals
h1e_cas, ecore = mycas.get_h1eff()  # One-electron integrals + core energy
h2e_cas = mycas.get_h2eff()         # Two-electron integrals

print("h2e_cas type:", type(h2e_cas))
print("h2e_cas shape:", h2e_cas.shape) # This result shows a 2D array, symmetric reduced form

norb = norb_active  

# "21" = norb*(norb+1)//2 for norb=6, so s4 symmetry - It was symmetrical reduced. Got into case with no symmetry
h2e_4d = ao2mo.restore(1, h2e_cas, norb) #Check this link (https://pyscf.org/pyscf_api_docs/pyscf.ao2mo.html)

print('Shape of h2e_4d:', h2e_4d.shape)  # Should be (norb, norb, norb, norb)


one_mo = h1e_cas
two_mo = h2e_4d
h2_phys = np.swapaxes(two_mo, 1, 3)  # Ensure the two-electron integrals are in the correct order
core_constant = np.array([ecore])

# Step 5: Continue with your original PennyLane code (unchanged)
H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, h2_phys)
H = qml.jordan_wigner(H_fermionic)
print("Hamiltonian H:", H)


converged SCF energy = -1655.10155370991  <S^2> = 6.0386361  2S+1 = 5.0154306
Atom list: Fe 0 0 0; S 0 0 1.826
Charge: 0
Spin: 4
Number of electrons: 42
CASSCF energy = -1655.37432225779
CASCI E = -1655.37432225779  E(CI) = -11.1006932117473  S^2 = 6.0000000
Hartree-Fock Ground State Energy: -1655.1015537099051
CASSCF Ground State Energy: -1655.3743222577903
h2e_cas type: <class 'numpy.ndarray'>
h2e_cas shape: (21, 21)
Shape of h2e_4d: (6, 6, 6, 6)
Hamiltonian H: (-1654.1365343987782+0j) * I(0) + (0.11642902671838198+0j) * Z(0) + (6.379872704345578e-07+0j) * (Y(0) @ Z(1) @ Y(2)) + (6.379872704345578e-07+0j) * (X(0) @ Z(1) @ X(2)) + (-7.862550676863911e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + (-7.862550676863911e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + -1.682681771697503e-16j * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + (0.013039803893742439+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + (0.013039803893742439+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @

In [9]:
A_code_CASSCF=-1655.37432225459
print('Dif. between Acode and PennyLane is', mycas.e_tot - A_code_CASSCF)

Dif. between Acode and PennyLane is 1.1907559382962063e-08


In [6]:
import pennylane as qml
X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

In [7]:
H = ((
    (-1654.1365343987782+0j) * I(0)
  + (0.11642902671838198+0j) * Z(0)
  + (6.379872704345578e-07+0j) * (Y(0) @ Z(1) @ Y(2))
  + (6.379872704345578e-07+0j) * (X(0) @ Z(1) @ X(2))
  + (-7.862550676863911e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
  + (-7.862550676863911e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4))
  + -1.682681771697503e-16j * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + (0.013039803893742439+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (0.013039803893742439+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + 1.682681771697503e-16j * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (0.008017830501243035+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (0.008017830501243035+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (1.3867406974691788e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (1.3867406974691788e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.1373620283043585+0j) * Z(2)
  + (0.10429184858679279+0j) * (Z(0) @ Z(2))
  + (-4.954206242150404e-06+0j) * (Y(2) @ Z(3) @ Y(4))
  + (-4.954206242150404e-06+0j) * (X(2) @ Z(3) @ X(4))
  + (-1.9445654976816007e-06+0j) * (Z(0) @ Y(2) @ Z(3) @ Y(4))
  + (-1.9445654976816007e-06+0j) * (Z(0) @ X(2) @ Z(3) @ X(4))
  + (2.9763439397507355e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (2.9763439397507355e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + (-5.182905059047801e-06+0j) * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (-5.182905059047801e-06+0j) * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + (-1.232225247728785e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-1.232225247728785e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-2.6508887444269636e-07+0j) * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-2.6508887444269636e-07+0j) * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (4.3674826463292863e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (4.3674826463292863e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-2.6735886171801985e-07+0j) * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-2.6735886171801985e-07+0j) * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.13735898149644893+0j) * Z(4)
  + (0.10430211516201834+0j) * (Z(0) @ Z(4))
  + (2.2739670283974926e-06+0j) * (Y(4) @ Z(5) @ Y(6))
  + (2.2739670283974926e-06+0j) * (X(4) @ Z(5) @ X(6))
  + (8.248046729097081e-06+0j) * (Z(0) @ Y(4) @ Z(5) @ Y(6))
  + (8.248046729097081e-06+0j) * (Z(0) @ X(4) @ Z(5) @ X(6))
  + (8.584966781114483e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (8.584966781114483e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (1.5950869357053617e-06+0j) * (Z(0) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (1.5950869357053617e-06+0j) * (Z(0) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-5.695864136586754e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-5.695864136586754e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.699863943826734e-07+0j) * (Z(0) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-1.699863943826734e-07+0j) * (Z(0) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.18552649598382548+0j) * Z(6)
  + (0.07414845792433955+0j) * (Z(0) @ Z(6))
  + (0.018902473647456952+0j) * (Y(6) @ Z(7) @ Y(8))
  + (0.018902473647456952+0j) * (X(6) @ Z(7) @ X(8))
  + (-0.005846881902536754+0j) * (Z(0) @ Y(6) @ Z(7) @ Y(8))
  + (-0.005846881902536754+0j) * (Z(0) @ X(6) @ Z(7) @ X(8))
  + (-1.1752452918857318e-05+0j) * (Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-1.1752452918857318e-05+0j) * (X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-3.485880402420099e-05+0j) * (Z(0) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-3.485880402420099e-05+0j) * (Z(0) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.048067985919588996+0j) * Z(8)
  + (0.05704912227141002+0j) * (Z(0) @ Z(8))
  + (1.2408573319595864e-05+0j) * (Y(8) @ Z(9) @ Y(10))
  + (1.2408573319595864e-05+0j) * (X(8) @ Z(9) @ X(10))
  + (-4.5592596601145836e-06+0j) * (Z(0) @ Y(8) @ Z(9) @ Y(10))
  + (-4.5592596601145836e-06+0j) * (Z(0) @ X(8) @ Z(9) @ X(10))
  + (0.12640039277311188+0j) * Z(10)
  + (0.10232591873617587+0j) * (Z(0) @ Z(10))
  + (0.11642902671838203+0j) * Z(1)
  + (0.09565267217372445+0j) * (Z(0) @ Z(1))
  + (1.5314567054220143e-06+0j) * (Y(0) @ Y(2))
  + (1.5314567054220143e-06+0j) * (X(0) @ X(2))
  + (-2.5010505561813555e-06+0j) * (Y(0) @ Z(2) @ Z(3) @ Y(4))
  + (-2.5010505561813555e-06+0j) * (X(0) @ Z(2) @ Z(3) @ X(4))
  + (-0.006772841908450757+0j) * (Y(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (-0.006772841908450757+0j) * (X(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + (-0.012896228128588146+0j) * (Y(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-0.012896228128588146+0j) * (X(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (1.2292221859520098e-05+0j) * (Y(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (1.2292221859520098e-05+0j) * (X(0) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (6.379872704345611e-07+0j) * (Y(1) @ Z(2) @ Y(3))
  + (1.5314567054220145e-06+0j) * (Z(0) @ Y(1) @ Z(2) @ Y(3))
  + (6.379872704345611e-07+0j) * (X(1) @ Z(2) @ X(3))
  + (1.5314567054220145e-06+0j) * (Z(0) @ X(1) @ Z(2) @ X(3))
  + (0.002250666097737803+0j) * (Y(0) @ X(1) @ X(2) @ Y(3))
  + (-0.002250666097737803+0j) * (Y(0) @ Y(1) @ X(2) @ X(3))
  + (-0.002250666097737803+0j) * (X(0) @ X(1) @ Y(2) @ Y(3))
  + (0.002250666097737803+0j) * (X(0) @ Y(1) @ Y(2) @ X(3))
  + (5.79303961720545e-07+0j) * (Y(0) @ X(1) @ X(3) @ Y(4))
  + (5.79303961720545e-07+0j) * (Y(0) @ Y(1) @ Y(3) @ Y(4))
  + (5.79303961720545e-07+0j) * (X(0) @ X(1) @ X(3) @ X(4))
  + (5.79303961720545e-07+0j) * (X(0) @ Y(1) @ Y(3) @ X(4))
  + (5.049422274168417e-06+0j) * (Y(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ Y(6))
  + (5.049422274168417e-06+0j) * (Y(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ Y(6))
  + (5.049422274168417e-06+0j) * (X(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ X(6))
  + (5.049422274168417e-06+0j) * (X(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ X(6))
  + (-1.0480250294211228e-06+0j) * (Y(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-1.0480250294211228e-06+0j) * (Y(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-1.0480250294211228e-06+0j) * (X(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-1.0480250294211228e-06+0j) * (X(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (1.2626206849948594e-07+0j) * (Y(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (1.2626206849948594e-07+0j) * (Y(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (1.2626206849948594e-07+0j) * (X(0) @ X(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (1.2626206849948594e-07+0j) * (X(0) @ Y(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-7.862550676863896e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5))
  + (-2.5010505561813555e-06+0j) * (Z(0) @ Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5))
  + (-7.862550676863896e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5))
  + (-2.5010505561813555e-06+0j) * (Z(0) @ X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5))
  + (-5.793039617205449e-07+0j) * (Y(0) @ X(1) @ X(2) @ Z(3) @ Z(4) @ Y(5))
  + (5.793039617205449e-07+0j) * (Y(0) @ Y(1) @ X(2) @ Z(3) @ Z(4) @ X(5))
  + (5.793039617205449e-07+0j) * (X(0) @ X(1) @ Y(2) @ Z(3) @ Z(4) @ Y(5))
  + (-5.793039617205449e-07+0j) * (X(0) @ Y(1) @ Y(2) @ Z(3) @ Z(4) @ X(5))
  + (0.0022477948302346185+0j) * (Y(0) @ X(1) @ X(4) @ Y(5))
  + (-0.0022477948302346185+0j) * (Y(0) @ Y(1) @ X(4) @ X(5))
  + (-0.0022477948302346185+0j) * (X(0) @ X(1) @ Y(4) @ Y(5))
  + (0.0022477948302346185+0j) * (X(0) @ Y(1) @ Y(4) @ X(5))
  + (-7.135402718854733e-06+0j) * (Y(0) @ X(1) @ X(5) @ Y(6))
  + (-7.135402718854733e-06+0j) * (Y(0) @ Y(1) @ Y(5) @ Y(6))
  + (-7.135402718854733e-06+0j) * (X(0) @ X(1) @ X(5) @ X(6))
  + (-7.135402718854733e-06+0j) * (X(0) @ Y(1) @ Y(5) @ X(6))
  + (1.3041468761104583e-06+0j) * (Y(0) @ X(1) @ X(5) @ Z(6) @ Z(7) @ Y(8))
  + (1.3041468761104583e-06+0j) * (Y(0) @ Y(1) @ Y(5) @ Z(6) @ Z(7) @ Y(8))
  + (1.3041468761104583e-06+0j) * (X(0) @ X(1) @ X(5) @ Z(6) @ Z(7) @ X(8))
  + (1.3041468761104583e-06+0j) * (X(0) @ Y(1) @ Y(5) @ Z(6) @ Z(7) @ X(8))
  + (1.2235618390895025e-07+0j) * (Y(0) @ X(1) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (1.2235618390895025e-07+0j) * (Y(0) @ Y(1) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (1.2235618390895025e-07+0j) * (X(0) @ X(1) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (1.2235618390895025e-07+0j) * (X(0) @ Y(1) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + -1.6544925152128798e-16j * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (0.013039803893742473+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (-0.006772841908450757+0j) * (Z(0) @ Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (0.013039803893742473+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (-0.006772841908450757+0j) * (Z(0) @ X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + 1.6544925152128798e-16j * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (-5.049422274168417e-06+0j) * (Y(0) @ X(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (5.049422274168417e-06+0j) * (Y(0) @ Y(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (5.049422274168417e-06+0j) * (X(0) @ X(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (-5.049422274168417e-06+0j) * (X(0) @ Y(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (7.135402718854733e-06+0j) * (Y(0) @ X(1) @ X(4) @ Z(5) @ Z(6) @ Y(7))
  + (-7.135402718854733e-06+0j) * (Y(0) @ Y(1) @ X(4) @ Z(5) @ Z(6) @ X(7))
  + (-7.135402718854733e-06+0j) * (X(0) @ X(1) @ Y(4) @ Z(5) @ Z(6) @ Y(7))
  + (7.135402718854733e-06+0j) * (X(0) @ Y(1) @ Y(4) @ Z(5) @ Z(6) @ X(7))
  + (0.034235367440510374+0j) * (Y(0) @ X(1) @ X(6) @ Y(7))
  + (-0.034235367440510374+0j) * (Y(0) @ Y(1) @ X(6) @ X(7))
  + (-0.034235367440510374+0j) * (X(0) @ X(1) @ Y(6) @ Y(7))
  + (0.034235367440510374+0j) * (X(0) @ Y(1) @ Y(6) @ X(7))
  + (0.007606054862134452+0j) * (Y(0) @ X(1) @ X(7) @ Y(8))
  + (0.007606054862134452+0j) * (Y(0) @ Y(1) @ Y(7) @ Y(8))
  + (0.007606054862134452+0j) * (X(0) @ X(1) @ X(7) @ X(8))
  + (0.007606054862134452+0j) * (X(0) @ Y(1) @ Y(7) @ X(8))
  + (3.47456213194859e-05+0j) * (Y(0) @ X(1) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (3.47456213194859e-05+0j) * (Y(0) @ Y(1) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (3.47456213194859e-05+0j) * (X(0) @ X(1) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (3.47456213194859e-05+0j) * (X(0) @ Y(1) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (0.008017830501243037+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-0.012896228128588145+0j) * (Z(0) @ Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (0.008017830501243037+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-0.012896228128588145+0j) * (Z(0) @ X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (1.0480250294211228e-06+0j) * (Y(0) @ X(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-1.0480250294211228e-06+0j) * (Y(0) @ Y(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-1.0480250294211228e-06+0j) * (X(0) @ X(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (1.0480250294211228e-06+0j) * (X(0) @ Y(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-1.3041468761104583e-06+0j) * (Y(0) @ X(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (1.3041468761104583e-06+0j) * (Y(0) @ Y(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (1.3041468761104583e-06+0j) * (X(0) @ X(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-1.3041468761104583e-06+0j) * (X(0) @ Y(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-0.007606054862134452+0j) * (Y(0) @ X(1) @ X(6) @ Z(7) @ Z(8) @ Y(9))
  + (0.007606054862134452+0j) * (Y(0) @ Y(1) @ X(6) @ Z(7) @ Z(8) @ X(9))
  + (0.007606054862134452+0j) * (X(0) @ X(1) @ Y(6) @ Z(7) @ Z(8) @ Y(9))
  + (-0.007606054862134452+0j) * (X(0) @ Y(1) @ Y(6) @ Z(7) @ Z(8) @ X(9))
  + (0.009657716169304434+0j) * (Y(0) @ X(1) @ X(8) @ Y(9))
  + (-0.009657716169304434+0j) * (Y(0) @ Y(1) @ X(8) @ X(9))
  + (-0.009657716169304434+0j) * (X(0) @ X(1) @ Y(8) @ Y(9))
  + (0.009657716169304434+0j) * (X(0) @ Y(1) @ Y(8) @ X(9))
  + (-8.08327051725109e-06+0j) * (Y(0) @ X(1) @ X(9) @ Y(10))
  + (-8.08327051725109e-06+0j) * (Y(0) @ Y(1) @ Y(9) @ Y(10))
  + (-8.08327051725109e-06+0j) * (X(0) @ X(1) @ X(9) @ X(10))
  + (-8.08327051725109e-06+0j) * (X(0) @ Y(1) @ Y(9) @ X(10))
  + (1.3867406974691863e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.2292221859520098e-05+0j) * (Z(0) @ Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.3867406974691863e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (1.2292221859520098e-05+0j) * (Z(0) @ X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.2626206849948594e-07+0j) * (Y(0) @ X(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.2626206849948594e-07+0j) * (Y(0) @ Y(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (1.2626206849948594e-07+0j) * (X(0) @ X(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.2626206849948594e-07+0j) * (X(0) @ Y(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.2235618390895025e-07+0j) * (Y(0) @ X(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.2235618390895025e-07+0j) * (Y(0) @ Y(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (1.2235618390895025e-07+0j) * (X(0) @ X(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.2235618390895025e-07+0j) * (X(0) @ Y(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-3.47456213194859e-05+0j) * (Y(0) @ X(1) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (3.47456213194859e-05+0j) * (Y(0) @ Y(1) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (3.47456213194859e-05+0j) * (X(0) @ X(1) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-3.47456213194859e-05+0j) * (X(0) @ Y(1) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (8.08327051725109e-06+0j) * (Y(0) @ X(1) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (-8.08327051725109e-06+0j) * (Y(0) @ Y(1) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (-8.08327051725109e-06+0j) * (X(0) @ X(1) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (8.08327051725109e-06+0j) * (X(0) @ Y(1) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (0.0031461213619085357+0j) * (Y(0) @ X(1) @ X(10) @ Y(11))
  + (-0.0031461213619085357+0j) * (Y(0) @ Y(1) @ X(10) @ X(11))
  + (-0.0031461213619085357+0j) * (X(0) @ X(1) @ Y(10) @ Y(11))
  + (0.0031461213619085357+0j) * (X(0) @ Y(1) @ Y(10) @ X(11))
  + (-1.189256880201914e-05+0j) * (Y(0) @ Z(1) @ Z(3) @ Y(4))
  + (-1.189256880201914e-05+0j) * (X(0) @ Z(1) @ Z(3) @ X(4))
  + (-0.05143476191748811+0j) * (Y(0) @ Z(1) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (-0.05143476191748811+0j) * (X(0) @ Z(1) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + (-0.0033701750786162893+0j) * (Y(0) @ Z(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-0.0033701750786162893+0j) * (X(0) @ Z(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-0.002840151434199984+0j) * (Y(0) @ Z(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.002840151434199984+0j) * (X(0) @ Z(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (8.042992531902197e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(4))
  + (8.042992531902197e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(4))
  + (-2.6504163293890345e-06+0j) * (Y(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ Y(6))
  + (-2.6519673446463496e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ Y(6))
  + (-1.5510152573147789e-09+0j) * (Y(0) @ Z(1) @ Y(2) @ X(4) @ Z(5) @ X(6))
  + (-1.5510152573147789e-09+0j) * (X(0) @ Z(1) @ X(2) @ Y(4) @ Z(5) @ Y(6))
  + (-2.6519673446463496e-06+0j) * (X(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ X(6))
  + (-2.6504163293890345e-06+0j) * (X(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ X(6))
  + (-3.7074519229250254e-07+0j) * (Y(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-3.697940046199378e-07+0j) * (Y(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (9.51187672564755e-10+0j) * (Y(0) @ Z(1) @ Y(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (9.51187672564755e-10+0j) * (X(0) @ Z(1) @ X(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-3.697940046199378e-07+0j) * (X(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-3.7074519229250254e-07+0j) * (X(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (0.0008920493688948626+0j) * (Y(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.0008914823824286827+0j) * (Y(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-5.669864661798679e-07+0j) * (Y(0) @ Z(1) @ Y(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-5.669864661798679e-07+0j) * (X(0) @ Z(1) @ X(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.0008914823824286827+0j) * (X(0) @ Z(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.0008920493688948626+0j) * (X(0) @ Z(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (8.922554909324522e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(6))
  + (8.922554909324522e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(6))
  + (-1.0740930716360443e-07+0j) * (Y(0) @ Z(1) @ X(2) @ X(6) @ Z(7) @ Y(8))
  + (-1.1514041160251484e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Y(6) @ Z(7) @ Y(8))
  + (-1.043994808861544e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ X(6) @ Z(7) @ X(8))
  + (-1.043994808861544e-06+0j) * (X(0) @ Z(1) @ X(2) @ Y(6) @ Z(7) @ Y(8))
  + (-1.1514041160251484e-06+0j) * (X(0) @ Z(1) @ X(2) @ X(6) @ Z(7) @ X(8))
  + (-1.0740930716360443e-07+0j) * (X(0) @ Z(1) @ Y(2) @ Y(6) @ Z(7) @ X(8))
  + (-1.9921685508420358e-07+0j) * (Y(0) @ Z(1) @ X(2) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-7.482906647794253e-07+0j) * (Y(0) @ Z(1) @ Y(2) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-5.490738096952216e-07+0j) * (Y(0) @ Z(1) @ Y(2) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-5.490738096952216e-07+0j) * (X(0) @ Z(1) @ X(2) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-7.482906647794253e-07+0j) * (X(0) @ Z(1) @ X(2) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.9921685508420358e-07+0j) * (X(0) @ Z(1) @ Y(2) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (2.57261277471324e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(8))
  + (2.57261277471324e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(8))
  + (-5.8946036810601414e-08+0j) * (Y(0) @ Z(1) @ X(2) @ X(8) @ Z(9) @ Y(10))
  + (-1.4084316502240574e-07+0j) * (Y(0) @ Z(1) @ Y(2) @ Y(8) @ Z(9) @ Y(10))
  + (-8.189712821180429e-08+0j) * (Y(0) @ Z(1) @ Y(2) @ X(8) @ Z(9) @ X(10))
  + (-8.189712821180429e-08+0j) * (X(0) @ Z(1) @ X(2) @ Y(8) @ Z(9) @ Y(10))
  + (-1.4084316502240574e-07+0j) * (X(0) @ Z(1) @ X(2) @ X(8) @ Z(9) @ X(10))
  + (-5.8946036810601414e-08+0j) * (X(0) @ Z(1) @ Y(2) @ Y(8) @ Z(9) @ X(10))
  + (8.683108875713097e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(10))
  + (8.683108875713097e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(10))
  + (0.13736202830435823+0j) * Z(3)
  + (0.10654251468453059+0j) * (Z(0) @ Z(3))
  + (9.557398348723318e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(3))
  + (9.557398348723318e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(3))
  + (-1.2520219744696908e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(4))
  + (-1.2520219744696908e-05+0j) * (X(0) @ Z(1) @ Z(2) @ X(4))
  + (-0.05329257158285347+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(4) @ Z(5) @ Y(6))
  + (-0.05329257158285347+0j) * (X(0) @ Z(1) @ Z(2) @ Z(4) @ Z(5) @ X(6))
  + (-0.0031956774059587936+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-0.0031956774059587936+0j) * (X(0) @ Z(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-0.0020204791262047584+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0020204791262047584+0j) * (X(0) @ Z(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-4.954206242150404e-06+0j) * (Y(3) @ Z(4) @ Y(5))
  + (-1.365261535961056e-06+0j) * (Z(0) @ Y(3) @ Z(4) @ Y(5))
  + (-4.954206242150404e-06+0j) * (X(3) @ Z(4) @ X(5))
  + (-1.365261535961056e-06+0j) * (Z(0) @ X(3) @ Z(4) @ X(5))
  + (6.276509426777666e-07+0j) * (Y(0) @ Y(3) @ Z(4) @ Y(5) @ Z(1) @ Y(2))
  + (6.276509426777666e-07+0j) * (Y(0) @ X(3) @ Z(4) @ X(5) @ Z(1) @ Y(2))
  + (6.276509426777666e-07+0j) * (X(0) @ Y(3) @ Z(4) @ Y(5) @ Z(1) @ X(2))
  + (6.276509426777666e-07+0j) * (X(0) @ X(3) @ Z(4) @ X(5) @ Z(1) @ X(2))
  + (-5.015356779050896e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(4) @ Y(5))
  + (5.015356779050896e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ X(4) @ X(5))
  + (5.015356779050896e-07+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ Y(4) @ Y(5))
  + (-5.015356779050896e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(4) @ X(5))
  + (-1.7075154549362235e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ Y(6))
  + (-1.7075154549362235e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ Y(6))
  + (-1.7075154549362235e-06+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ X(6))
  + (-1.7075154549362235e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ X(6))
  + (-7.656833997037078e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Y(8))
  + (-7.656833997037078e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Y(8))
  + (-7.656833997037078e-08+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ X(8))
  + (-7.656833997037078e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ X(8))
  + (0.000640658413980919+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.000640658413980919+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.000640658413980919+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.000640658413980919+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (2.976343939750738e-06+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (-1.3348278487938507e-07+0j) * (Z(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (2.976343939750738e-06+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (-1.3348278487938507e-07+0j) * (Z(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (0.001857809665365363+0j) * (Y(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(1) @ Y(2))
  + (0.001857809665365363+0j) * (Y(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(1) @ Y(2))
  + (0.001857809665365363+0j) * (X(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(1) @ X(2))
  + (0.001857809665365363+0j) * (X(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(1) @ X(2))
  + (-9.42900874452811e-07+0j) * (Y(0) @ X(3) @ X(4) @ Z(5) @ Z(6) @ Y(7) @ Z(1) @ Z(2))
  + (9.42900874452811e-07+0j) * (Y(0) @ Y(3) @ X(4) @ Z(5) @ Z(6) @ X(7) @ Z(1) @ Z(2))
  + (9.42900874452811e-07+0j) * (X(0) @ X(3) @ Y(4) @ Z(5) @ Z(6) @ Y(7) @ Z(1) @ Z(2))
  + (-9.42900874452811e-07+0j) * (X(0) @ Y(3) @ Y(4) @ Z(5) @ Z(6) @ X(7) @ Z(1) @ Z(2))
  + (2.8206751641116322e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(6) @ Y(7))
  + (-2.8206751641116322e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ X(6) @ X(7))
  + (-2.8206751641116322e-06+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ Y(6) @ Y(7))
  + (2.8206751641116322e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(6) @ X(7))
  + (8.830236902708366e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(7) @ Y(8))
  + (8.830236902708366e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ Y(7) @ Y(8))
  + (8.830236902708366e-07+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ X(7) @ X(8))
  + (8.830236902708366e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(7) @ X(8))
  + (-3.370135787139709e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (-3.370135787139709e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (-3.370135787139709e-07+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (-3.370135787139709e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.232225247728785e-06+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-1.3131139038638193e-06+0j) * (Z(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-1.232225247728785e-06+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-1.3131139038638193e-06+0j) * (Z(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-0.00017449767265749597+0j) * (Y(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ Y(2))
  + (-0.00017449767265749597+0j) * (Y(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ Y(2))
  + (-0.00017449767265749597+0j) * (X(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ X(2))
  + (-0.00017449767265749597+0j) * (X(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ X(2))
  + (-2.941768523221318e-07+0j) * (Y(0) @ X(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ Z(2))
  + (2.941768523221318e-07+0j) * (Y(0) @ Y(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ Z(2))
  + (2.941768523221318e-07+0j) * (X(0) @ X(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ Z(2))
  + (-2.941768523221318e-07+0j) * (X(0) @ Y(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ Z(2))
  + (-9.90432997434441e-07+0j) * (Y(0) @ X(3) @ X(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ Z(2))
  + (9.90432997434441e-07+0j) * (Y(0) @ Y(3) @ X(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ Z(2))
  + (9.90432997434441e-07+0j) * (X(0) @ X(3) @ Y(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ Z(2))
  + (-9.90432997434441e-07+0j) * (X(0) @ Y(3) @ Y(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ Z(2))
  + (7.805126792199789e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(8) @ Y(9))
  + (-7.805126792199789e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ X(8) @ X(9))
  + (-7.805126792199789e-07+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ Y(8) @ Y(9))
  + (7.805126792199789e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(8) @ X(9))
  + (-1.012573148728939e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(9) @ Y(10))
  + (-1.012573148728939e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ Y(9) @ Y(10))
  + (-1.012573148728939e-07+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ X(9) @ X(10))
  + (-1.012573148728939e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(9) @ X(10))
  + (4.3674826463292885e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.4109679321853394e-07+0j) * (Z(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (4.3674826463292885e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.4109679321853394e-07+0j) * (Z(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.000819672307995226+0j) * (Y(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Y(2))
  + (-0.000819672307995226+0j) * (Y(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Y(2))
  + (-0.000819672307995226+0j) * (X(0) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ X(2))
  + (-0.000819672307995226+0j) * (X(0) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ X(2))
  + (0.00025139095491394354+0j) * (Y(0) @ X(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2))
  + (-0.00025139095491394354+0j) * (Y(0) @ Y(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2))
  + (-0.00025139095491394354+0j) * (X(0) @ X(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2))
  + (0.00025139095491394354+0j) * (X(0) @ Y(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2))
  + (1.377967236297673e-07+0j) * (Y(0) @ X(3) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2))
  + (-1.377967236297673e-07+0j) * (Y(0) @ Y(3) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2))
  + (-1.377967236297673e-07+0j) * (X(0) @ X(3) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2))
  + (1.377967236297673e-07+0j) * (X(0) @ Y(3) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2))
  + (4.231127806229248e-08+0j) * (Y(0) @ X(3) @ X(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2))
  + (-4.231127806229248e-08+0j) * (Y(0) @ Y(3) @ X(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2))
  + (-4.231127806229248e-08+0j) * (X(0) @ X(3) @ Y(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2))
  + (4.231127806229248e-08+0j) * (X(0) @ Y(3) @ Y(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2))
  + (-5.527425818816837e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ X(3) @ X(10) @ Y(11))
  + (5.527425818816837e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Y(3) @ X(10) @ X(11))
  + (5.527425818816837e-07+0j) * (X(0) @ Z(1) @ Z(2) @ X(3) @ Y(10) @ Y(11))
  + (-5.527425818816837e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Y(3) @ Y(10) @ X(11))
  + (-0.05144570259023998+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(5) @ Y(6))
  + (-0.05144570259023998+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(5) @ X(6))
  + (-0.0033716933276748155+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-0.0033716933276748155+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (0.002950247188821601+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.002950247188821601+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.3853960588383758e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(6))
  + (-1.3853960588383758e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(6))
  + (-8.070096674074511e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Y(8))
  + (1.5368154325893712e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Y(8))
  + (2.3438250999968223e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ X(6) @ Z(7) @ X(8))
  + (2.3438250999968223e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Y(6) @ Z(7) @ Y(8))
  + (1.5368154325893712e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ X(8))
  + (-8.070096674074511e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ X(8))
  + (-1.3547393093620481e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-6.09663819969118e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-4.7418988903291327e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-4.7418988903291327e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-6.09663819969118e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.3547393093620481e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-3.9898308887889304e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(8))
  + (-3.9898308887889304e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(8))
  + (-1.0087464556097522e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(8) @ Z(9) @ Y(10))
  + (-1.369110775400953e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(8) @ Z(9) @ Y(10))
  + (-3.603643197912009e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ X(8) @ Z(9) @ X(10))
  + (-3.603643197912009e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Y(8) @ Z(9) @ Y(10))
  + (-1.369110775400953e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(8) @ Z(9) @ X(10))
  + (-1.0087464556097522e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(8) @ Z(9) @ X(10))
  + (-1.2599081894237662e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(10))
  + (-1.2599081894237662e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(10))
  + (0.13735898149644893+0j) * Z(5)
  + (0.10654990999225297+0j) * (Z(0) @ Z(5))
  + (8.544528209807287e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(5))
  + (8.544528209807287e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(5))
  + (-1.3842259855013355e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(5))
  + (-1.3842259855013355e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(5))
  + (-0.053299979303300576+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(6))
  + (-0.053299979303300576+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(6))
  + (-0.0031957805885567793+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ Y(8))
  + (-0.0031957805885567793+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ X(8))
  + (0.0021394350862313644+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.0021394350862313644+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (2.2739670283974956e-06+0j) * (Y(5) @ Z(6) @ Y(7))
  + (1.112644010242348e-06+0j) * (Z(0) @ Y(5) @ Z(6) @ Y(7))
  + (2.2739670283974956e-06+0j) * (X(5) @ Z(6) @ X(7))
  + (1.112644010242348e-06+0j) * (Z(0) @ X(5) @ Z(6) @ X(7))
  + (-9.444518897101256e-07+0j) * (Y(0) @ Y(5) @ Z(6) @ Y(7) @ Z(1) @ Y(2))
  + (-9.444518897101256e-07+0j) * (Y(0) @ X(5) @ Z(6) @ X(7) @ Z(1) @ Y(2))
  + (-9.444518897101256e-07+0j) * (X(0) @ Y(5) @ Z(6) @ Y(7) @ Z(1) @ X(2))
  + (-9.444518897101256e-07+0j) * (X(0) @ X(5) @ Z(6) @ X(7) @ Z(1) @ X(2))
  + (0.0018542767130606034+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(5) @ Z(6) @ Y(7))
  + (0.0018542767130606034+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ X(5) @ Z(6) @ X(7))
  + (0.0018542767130606034+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Y(5) @ Z(6) @ Y(7))
  + (0.0018542767130606034+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(5) @ Z(6) @ X(7))
  + (-4.989857748451018e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(6) @ Y(7))
  + (4.989857748451018e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(6) @ X(7))
  + (4.989857748451018e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(6) @ Y(7))
  + (-4.989857748451018e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(6) @ X(7))
  + (-1.1665999509906146e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Y(8))
  + (-1.1665999509906146e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Y(8))
  + (-1.1665999509906146e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ X(8))
  + (-1.1665999509906146e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ X(8))
  + (-2.762169875580479e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (-2.762169875580479e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (-2.762169875580479e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (-2.762169875580479e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (8.58496678111449e-07+0j) * (Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (2.89923381181582e-06+0j) * (Z(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (8.58496678111449e-07+0j) * (X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (2.89923381181582e-06+0j) * (Z(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-2.93225664649567e-07+0j) * (Y(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ Y(2))
  + (-2.93225664649567e-07+0j) * (Y(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ Y(2))
  + (-2.93225664649567e-07+0j) * (X(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ X(2))
  + (-2.93225664649567e-07+0j) * (X(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ X(2))
  + (-0.00017591273911803616+0j) * (Y(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
  + (-0.00017591273911803616+0j) * (Y(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
  + (-0.00017591273911803616+0j) * (X(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(1) @ Z(2) @ Z(3) @ X(4))
  + (-0.00017591273911803616+0j) * (X(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(1) @ Z(2) @ Z(3) @ X(4))
  + (3.5959028358316347e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(6) @ Z(7) @ Z(8) @ Y(9))
  + (-3.5959028358316347e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(6) @ Z(7) @ Z(8) @ X(9))
  + (-3.5959028358316347e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(6) @ Z(7) @ Z(8) @ Y(9))
  + (3.5959028358316347e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(6) @ Z(7) @ Z(8) @ X(9))
  + (-1.6081416347229119e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(8) @ Y(9))
  + (1.6081416347229119e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(8) @ X(9))
  + (1.6081416347229119e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(8) @ Y(9))
  + (-1.6081416347229119e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(8) @ X(9))
  + (-6.683321744608681e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(9) @ Y(10))
  + (-6.683321744608681e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(9) @ Y(10))
  + (-6.683321744608681e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(9) @ X(10))
  + (-6.683321744608681e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(9) @ X(10))
  + (-5.695864136586753e-07+0j) * (Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-4.763021047372312e-08+0j) * (Z(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-5.695864136586753e-07+0j) * (X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-4.763021047372312e-08+0j) * (Z(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.00025082396844776367+0j) * (Y(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Y(2))
  + (0.00025082396844776367+0j) * (Y(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Y(2))
  + (0.00025082396844776367+0j) * (X(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ X(2))
  + (0.00025082396844776367+0j) * (X(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ X(2))
  + (0.0008108121025902371+0j) * (Y(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
  + (0.0008108121025902371+0j) * (Y(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
  + (0.0008108121025902371+0j) * (X(0) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2) @ Z(3) @ X(4))
  + (0.0008108121025902371+0j) * (X(0) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2) @ Z(3) @ X(4))
  + (1.407430566218431e-07+0j) * (Y(0) @ X(5) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2) @ Z(3) @ Z(4))
  + (-1.407430566218431e-07+0j) * (Y(0) @ Y(5) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2) @ Z(3) @ Z(4))
  + (-1.407430566218431e-07+0j) * (X(0) @ X(5) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2) @ Z(3) @ Z(4))
  + (1.407430566218431e-07+0j) * (X(0) @ Y(5) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2) @ Z(3) @ Z(4))
  + (-3.404142811488842e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (3.404142811488842e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (3.404142811488842e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (-3.404142811488842e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (7.347550678322491e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(10) @ Y(11))
  + (-7.347550678322491e-07+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(10) @ X(11))
  + (-7.347550678322491e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(10) @ Y(11))
  + (7.347550678322491e-07+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(10) @ X(11))
  + (0.0037272400076382452+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Y(8))
  + (0.0037272400076382452+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ X(8))
  + (5.68898190262677e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (5.68898190262677e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-0.01508988367698451+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(8))
  + (-0.01508988367698451+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(8))
  + (8.22799995475387e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ Y(10))
  + (1.1118289109801853e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ Y(10))
  + (2.890289155047985e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(8) @ Z(9) @ X(10))
  + (2.890289155047985e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(8) @ Z(9) @ Y(10))
  + (1.1118289109801853e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ X(10))
  + (8.22799995475387e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ X(10))
  + (-0.050866803551057625+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(10))
  + (-0.050866803551057625+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(10))
  + (0.18552649598382542+0j) * Z(7)
  + (0.10838382536484995+0j) * (Z(0) @ Z(7))
  + (6.1018797452128895e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(7))
  + (6.1018797452128895e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(7))
  + (-8.86410283993274e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(7))
  + (-8.86410283993274e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(7))
  + (-0.03934646448990639+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(7))
  + (-0.03934646448990639+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(7))
  + (-0.009537445350450844+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(8))
  + (-0.009537445350450844+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(8))
  + (4.034512916619667e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ Y(10))
  + (4.034512916619667e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ X(10))
  + (0.018902473647456938+0j) * (Y(7) @ Z(8) @ Y(9))
  + (0.0017591729595976973+0j) * (Z(0) @ Y(7) @ Z(8) @ Y(9))
  + (0.018902473647456938+0j) * (X(7) @ Z(8) @ X(9))
  + (0.0017591729595976973+0j) * (Z(0) @ X(7) @ Z(8) @ X(9))
  + (-2.0344278062959845e-06+0j) * (Y(0) @ Y(7) @ Z(8) @ Y(9) @ Z(1) @ Y(2))
  + (-2.0344278062959845e-06+0j) * (Y(0) @ X(7) @ Z(8) @ X(9) @ Z(1) @ Y(2))
  + (-2.0344278062959845e-06+0j) * (X(0) @ Y(7) @ Z(8) @ Y(9) @ Z(1) @ X(2))
  + (-2.0344278062959845e-06+0j) * (X(0) @ X(7) @ Z(8) @ X(9) @ Z(1) @ X(2))
  + (2.7034153835799853e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(7) @ Z(8) @ Y(9))
  + (2.7034153835799853e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ X(7) @ Z(8) @ X(9))
  + (2.7034153835799853e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Y(7) @ Z(8) @ Y(9))
  + (2.7034153835799853e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(7) @ Z(8) @ X(9))
  + (0.013264685358089088+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(7) @ Z(8) @ Y(9))
  + (0.013264685358089088+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(7) @ Z(8) @ X(9))
  + (0.013264685358089088+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(7) @ Z(8) @ Y(9))
  + (0.013264685358089088+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(7) @ Z(8) @ X(9))
  + (-0.004561593662947706+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(8) @ Y(9))
  + (0.004561593662947706+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(8) @ X(9))
  + (0.004561593662947706+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(8) @ Y(9))
  + (-0.004561593662947706+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(8) @ X(9))
  + (1.2337859629559085e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ Y(10))
  + (1.2337859629559085e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ Y(10))
  + (1.2337859629559085e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ X(10))
  + (1.2337859629559085e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ X(10))
  + (-1.1752452918856974e-05+0j) * (Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.1318270471508736e-07+0j) * (Z(0) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.1752452918856974e-05+0j) * (X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.1318270471508736e-07+0j) * (Z(0) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-4.112770860654544e-07+0j) * (Y(0) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Y(2))
  + (-4.112770860654544e-07+0j) * (Y(0) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Y(2))
  + (-4.112770860654544e-07+0j) * (X(0) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ X(2))
  + (-4.112770860654544e-07+0j) * (X(0) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ X(2))
  + (-3.334468324110703e-07+0j) * (Y(0) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
  + (-3.334468324110703e-07+0j) * (Y(0) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2) @ Z(3) @ Y(4))
  + (-3.334468324110703e-07+0j) * (X(0) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(1) @ Z(2) @ Z(3) @ X(4))
  + (-3.334468324110703e-07+0j) * (X(0) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(1) @ Z(2) @ Z(3) @ X(4))
  + (1.6544689860071028e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.6544689860071028e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (1.6544689860071028e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.6544689860071028e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-4.109859674805216e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (4.109859674805216e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (4.109859674805216e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (-4.109859674805216e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (0.003977948028580935+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(10) @ Y(11))
  + (-0.003977948028580935+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(10) @ X(11))
  + (-0.003977948028580935+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(10) @ Y(11))
  + (0.003977948028580935+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(10) @ X(11))
  + (1.747016111646224e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ Y(10))
  + (1.747016111646224e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ X(10))
  + (-0.0011212419840400235+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(10))
  + (-0.0011212419840400235+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(10))
  + (0.04806798591958895+0j) * Z(9)
  + (0.06670683844071446+0j) * (Z(0) @ Z(9))
  + (1.792100095493261e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(9))
  + (1.792100095493261e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(9))
  + (-2.3816892540660184e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(9))
  + (-2.3816892540660184e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(9))
  + (-0.010528290014036804+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(9))
  + (-0.010528290014036804+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(9))
  + (0.0008869981212886137+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(9))
  + (0.0008869981212886137+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(9))
  + (1.1617641017697768e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(10))
  + (1.1617641017697768e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(10))
  + (1.2408573319595949e-05+0j) * (Y(9) @ Z(10) @ Y(11))
  + (-1.2642530177365674e-05+0j) * (Z(0) @ Y(9) @ Z(10) @ Y(11))
  + (1.2408573319595949e-05+0j) * (X(9) @ Z(10) @ X(11))
  + (-1.2642530177365674e-05+0j) * (Z(0) @ X(9) @ Z(10) @ X(11))
  + (-3.958585014951183e-08+0j) * (Y(0) @ Y(9) @ Z(10) @ Y(11) @ Z(1) @ Y(2))
  + (-3.958585014951183e-08+0j) * (Y(0) @ X(9) @ Z(10) @ X(11) @ Z(1) @ Y(2))
  + (-3.958585014951183e-08+0j) * (X(0) @ Y(9) @ Z(10) @ Y(11) @ Z(1) @ X(2))
  + (-3.958585014951183e-08+0j) * (X(0) @ X(9) @ Z(10) @ X(11) @ Z(1) @ X(2))
  + (-7.007786009400852e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Y(9) @ Z(10) @ Y(11))
  + (-7.007786009400852e-08+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ X(9) @ Z(10) @ X(11))
  + (-7.007786009400852e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Y(9) @ Z(10) @ Y(11))
  + (-7.007786009400852e-08+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ X(9) @ Z(10) @ X(11))
  + (-1.2195705197572306e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(9) @ Z(10) @ Y(11))
  + (-1.2195705197572306e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(9) @ Z(10) @ X(11))
  + (-1.2195705197572306e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(9) @ Z(10) @ Y(11))
  + (-1.2195705197572306e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(9) @ Z(10) @ X(11))
  + (5.852520098764472e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(9) @ Z(10) @ Y(11))
  + (5.852520098764472e-06+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ X(9) @ Z(10) @ X(11))
  + (5.852520098764472e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Y(9) @ Z(10) @ Y(11))
  + (5.852520098764472e-06+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(9) @ Z(10) @ X(11))
  + (-0.0003207745359311322+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ X(10) @ Y(11))
  + (0.0003207745359311322+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ X(10) @ X(11))
  + (0.0003207745359311322+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Y(10) @ Y(11))
  + (-0.0003207745359311322+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Y(10) @ X(11))
  + (0.12640039277311188+0j) * Z(11)
  + (0.10547204009808439+0j) * (Z(0) @ Z(11))
  + (9.235851457594781e-06+0j) * (Y(0) @ Z(1) @ Y(2) @ Z(11))
  + (9.235851457594781e-06+0j) * (X(0) @ Z(1) @ X(2) @ Z(11))
  + (-1.3333836962069914e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4) @ Z(11))
  + (-1.3333836962069914e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4) @ Z(11))
  + (-0.054844751579638557+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(11))
  + (-0.054844751579638557+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(11))
  + (-0.000800467448108891+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(11))
  + (-0.000800467448108891+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(11))
  + (7.286853052733687e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(11))
  + (7.286853052733687e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(11))
  + (0.10429184858679279+0j) * (Z(1) @ Z(3))
  + (-1.9445654976816007e-06+0j) * (Z(1) @ Y(3) @ Z(4) @ Y(5))
  + (-1.9445654976816007e-06+0j) * (Z(1) @ X(3) @ Z(4) @ X(5))
  + (-5.182905059047801e-06+0j) * (Z(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (-5.182905059047801e-06+0j) * (Z(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (-2.6508887444269636e-07+0j) * (Z(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-2.6508887444269636e-07+0j) * (Z(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-2.6735886171801985e-07+0j) * (Z(1) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-2.6735886171801985e-07+0j) * (Z(1) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.10430211516201834+0j) * (Z(1) @ Z(5))
  + (8.248046729097081e-06+0j) * (Z(1) @ Y(5) @ Z(6) @ Y(7))
  + (8.248046729097081e-06+0j) * (Z(1) @ X(5) @ Z(6) @ X(7))
  + (1.5950869357053617e-06+0j) * (Z(1) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (1.5950869357053617e-06+0j) * (Z(1) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-1.699863943826734e-07+0j) * (Z(1) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.699863943826734e-07+0j) * (Z(1) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.07414845792433955+0j) * (Z(1) @ Z(7))
  + (-0.005846881902536754+0j) * (Z(1) @ Y(7) @ Z(8) @ Y(9))
  + (-0.005846881902536754+0j) * (Z(1) @ X(7) @ Z(8) @ X(9))
  + (-3.485880402420099e-05+0j) * (Z(1) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-3.485880402420099e-05+0j) * (Z(1) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.05704912227141002+0j) * (Z(1) @ Z(9))
  + (-4.5592596601145836e-06+0j) * (Z(1) @ Y(9) @ Z(10) @ Y(11))
  + (-4.5592596601145836e-06+0j) * (Z(1) @ X(9) @ Z(10) @ X(11))
  + (0.10232591873617587+0j) * (Z(1) @ Z(11))
  + (0.10654251468453059+0j) * (Z(1) @ Z(2))
  + (9.557398348723318e-06+0j) * (Y(1) @ Y(3))
  + (9.557398348723318e-06+0j) * (X(1) @ X(3))
  + (-1.2520219744696908e-05+0j) * (Y(1) @ Z(3) @ Z(4) @ Y(5))
  + (-1.2520219744696908e-05+0j) * (X(1) @ Z(3) @ Z(4) @ X(5))
  + (-0.05329257158285347+0j) * (Y(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (-0.05329257158285347+0j) * (X(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (-0.0031956774059587936+0j) * (Y(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-0.0031956774059587936+0j) * (X(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-0.0020204791262047584+0j) * (Y(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0020204791262047584+0j) * (X(1) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.365261535961056e-06+0j) * (Z(1) @ Y(2) @ Z(3) @ Y(4))
  + (-1.365261535961056e-06+0j) * (Z(1) @ X(2) @ Z(3) @ X(4))
  + (6.276509426777666e-07+0j) * (Y(1) @ X(2) @ X(3) @ Y(4))
  + (-6.276509426777666e-07+0j) * (Y(1) @ Y(2) @ X(3) @ X(4))
  + (-6.276509426777666e-07+0j) * (X(1) @ X(2) @ Y(3) @ Y(4))
  + (6.276509426777666e-07+0j) * (X(1) @ Y(2) @ Y(3) @ X(4))
  + (5.015356779050896e-07+0j) * (Y(1) @ X(2) @ X(4) @ Y(5))
  + (5.015356779050896e-07+0j) * (Y(1) @ Y(2) @ Y(4) @ Y(5))
  + (5.015356779050896e-07+0j) * (X(1) @ X(2) @ X(4) @ X(5))
  + (5.015356779050896e-07+0j) * (X(1) @ Y(2) @ Y(4) @ X(5))
  + (-1.7075154549362235e-06+0j) * (Y(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Y(7))
  + (-1.7075154549362235e-06+0j) * (Y(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Y(7))
  + (-1.7075154549362235e-06+0j) * (X(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ X(7))
  + (-1.7075154549362235e-06+0j) * (X(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ X(7))
  + (-7.656833997037078e-08+0j) * (Y(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-7.656833997037078e-08+0j) * (Y(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-7.656833997037078e-08+0j) * (X(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-7.656833997037078e-08+0j) * (X(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (0.000640658413980919+0j) * (Y(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.000640658413980919+0j) * (Y(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.000640658413980919+0j) * (X(1) @ X(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.000640658413980919+0j) * (X(1) @ Y(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.3348278487938507e-07+0j) * (Z(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (-1.3348278487938507e-07+0j) * (Z(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + (0.001857809665365363+0j) * (Y(1) @ X(2) @ X(3) @ Z(4) @ Z(5) @ Y(6))
  + (-0.001857809665365363+0j) * (Y(1) @ Y(2) @ X(3) @ Z(4) @ Z(5) @ X(6))
  + (-0.001857809665365363+0j) * (X(1) @ X(2) @ Y(3) @ Z(4) @ Z(5) @ Y(6))
  + (0.001857809665365363+0j) * (X(1) @ Y(2) @ Y(3) @ Z(4) @ Z(5) @ X(6))
  + (-9.42900874452811e-07+0j) * (Y(1) @ X(2) @ X(5) @ Y(6))
  + (9.42900874452811e-07+0j) * (Y(1) @ Y(2) @ X(5) @ X(6))
  + (9.42900874452811e-07+0j) * (X(1) @ X(2) @ Y(5) @ Y(6))
  + (-9.42900874452811e-07+0j) * (X(1) @ Y(2) @ Y(5) @ X(6))
  + (-2.8206751641116322e-06+0j) * (Y(1) @ X(2) @ X(6) @ Y(7))
  + (-2.8206751641116322e-06+0j) * (Y(1) @ Y(2) @ Y(6) @ Y(7))
  + (-2.8206751641116322e-06+0j) * (X(1) @ X(2) @ X(6) @ X(7))
  + (-2.8206751641116322e-06+0j) * (X(1) @ Y(2) @ Y(6) @ X(7))
  + (8.830236902708366e-07+0j) * (Y(1) @ X(2) @ X(6) @ Z(7) @ Z(8) @ Y(9))
  + (8.830236902708366e-07+0j) * (Y(1) @ Y(2) @ Y(6) @ Z(7) @ Z(8) @ Y(9))
  + (8.830236902708366e-07+0j) * (X(1) @ X(2) @ X(6) @ Z(7) @ Z(8) @ X(9))
  + (8.830236902708366e-07+0j) * (X(1) @ Y(2) @ Y(6) @ Z(7) @ Z(8) @ X(9))
  + (-3.370135787139709e-07+0j) * (Y(1) @ X(2) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-3.370135787139709e-07+0j) * (Y(1) @ Y(2) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-3.370135787139709e-07+0j) * (X(1) @ X(2) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-3.370135787139709e-07+0j) * (X(1) @ Y(2) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.3131139038638193e-06+0j) * (Z(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-1.3131139038638193e-06+0j) * (Z(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-0.00017449767265749597+0j) * (Y(1) @ X(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (0.00017449767265749597+0j) * (Y(1) @ Y(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (0.00017449767265749597+0j) * (X(1) @ X(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-0.00017449767265749597+0j) * (X(1) @ Y(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-2.941768523221318e-07+0j) * (Y(1) @ X(2) @ X(5) @ Z(6) @ Z(7) @ Y(8))
  + (2.941768523221318e-07+0j) * (Y(1) @ Y(2) @ X(5) @ Z(6) @ Z(7) @ X(8))
  + (2.941768523221318e-07+0j) * (X(1) @ X(2) @ Y(5) @ Z(6) @ Z(7) @ Y(8))
  + (-2.941768523221318e-07+0j) * (X(1) @ Y(2) @ Y(5) @ Z(6) @ Z(7) @ X(8))
  + (-9.90432997434441e-07+0j) * (Y(1) @ X(2) @ X(7) @ Y(8))
  + (9.90432997434441e-07+0j) * (Y(1) @ Y(2) @ X(7) @ X(8))
  + (9.90432997434441e-07+0j) * (X(1) @ X(2) @ Y(7) @ Y(8))
  + (-9.90432997434441e-07+0j) * (X(1) @ Y(2) @ Y(7) @ X(8))
  + (-7.805126792199789e-07+0j) * (Y(1) @ X(2) @ X(8) @ Y(9))
  + (-7.805126792199789e-07+0j) * (Y(1) @ Y(2) @ Y(8) @ Y(9))
  + (-7.805126792199789e-07+0j) * (X(1) @ X(2) @ X(8) @ X(9))
  + (-7.805126792199789e-07+0j) * (X(1) @ Y(2) @ Y(8) @ X(9))
  + (-1.012573148728939e-07+0j) * (Y(1) @ X(2) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.012573148728939e-07+0j) * (Y(1) @ Y(2) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.012573148728939e-07+0j) * (X(1) @ X(2) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.012573148728939e-07+0j) * (X(1) @ Y(2) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.4109679321853394e-07+0j) * (Z(1) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-1.4109679321853394e-07+0j) * (Z(1) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-0.000819672307995226+0j) * (Y(1) @ X(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.000819672307995226+0j) * (Y(1) @ Y(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.000819672307995226+0j) * (X(1) @ X(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.000819672307995226+0j) * (X(1) @ Y(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.00025139095491394354+0j) * (Y(1) @ X(2) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.00025139095491394354+0j) * (Y(1) @ Y(2) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-0.00025139095491394354+0j) * (X(1) @ X(2) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.00025139095491394354+0j) * (X(1) @ Y(2) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (1.377967236297673e-07+0j) * (Y(1) @ X(2) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (-1.377967236297673e-07+0j) * (Y(1) @ Y(2) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.377967236297673e-07+0j) * (X(1) @ X(2) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (1.377967236297673e-07+0j) * (X(1) @ Y(2) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (4.231127806229248e-08+0j) * (Y(1) @ X(2) @ X(9) @ Y(10))
  + (-4.231127806229248e-08+0j) * (Y(1) @ Y(2) @ X(9) @ X(10))
  + (-4.231127806229248e-08+0j) * (X(1) @ X(2) @ Y(9) @ Y(10))
  + (4.231127806229248e-08+0j) * (X(1) @ Y(2) @ Y(9) @ X(10))
  + (5.527425818816837e-07+0j) * (Y(1) @ X(2) @ X(10) @ Y(11))
  + (5.527425818816837e-07+0j) * (Y(1) @ Y(2) @ Y(10) @ Y(11))
  + (5.527425818816837e-07+0j) * (X(1) @ X(2) @ X(10) @ X(11))
  + (5.527425818816837e-07+0j) * (X(1) @ Y(2) @ Y(10) @ X(11))
  + (-1.189256880201914e-05+0j) * (Y(1) @ Z(2) @ Z(4) @ Y(5))
  + (-1.189256880201914e-05+0j) * (X(1) @ Z(2) @ Z(4) @ X(5))
  + (-0.05143476191748811+0j) * (Y(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (-0.05143476191748811+0j) * (X(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (-0.0033701750786162893+0j) * (Y(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-0.0033701750786162893+0j) * (X(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-0.002840151434199984+0j) * (Y(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.002840151434199984+0j) * (X(1) @ Z(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (8.042992531902197e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Z(5))
  + (8.042992531902197e-06+0j) * (X(1) @ Z(2) @ X(3) @ Z(5))
  + (-2.6504163293890345e-06+0j) * (Y(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Y(7))
  + (-2.6519673446463496e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Y(7))
  + (-1.5510152573147789e-09+0j) * (Y(1) @ Z(2) @ Y(3) @ X(5) @ Z(6) @ X(7))
  + (-1.5510152573147789e-09+0j) * (X(1) @ Z(2) @ X(3) @ Y(5) @ Z(6) @ Y(7))
  + (-2.6519673446463496e-06+0j) * (X(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ X(7))
  + (-2.6504163293890345e-06+0j) * (X(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ X(7))
  + (-3.7074519229250254e-07+0j) * (Y(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-3.697940046199378e-07+0j) * (Y(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (9.51187672564755e-10+0j) * (Y(1) @ Z(2) @ Y(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (9.51187672564755e-10+0j) * (X(1) @ Z(2) @ X(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-3.697940046199378e-07+0j) * (X(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-3.7074519229250254e-07+0j) * (X(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (0.0008920493688948626+0j) * (Y(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.0008914823824286827+0j) * (Y(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-5.669864661798679e-07+0j) * (Y(1) @ Z(2) @ Y(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-5.669864661798679e-07+0j) * (X(1) @ Z(2) @ X(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.0008914823824286827+0j) * (X(1) @ Z(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.0008920493688948626+0j) * (X(1) @ Z(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (8.922554909324522e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Z(7))
  + (8.922554909324522e-06+0j) * (X(1) @ Z(2) @ X(3) @ Z(7))
  + (-1.0740930716360443e-07+0j) * (Y(1) @ Z(2) @ X(3) @ X(7) @ Z(8) @ Y(9))
  + (-1.1514041160251484e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Y(7) @ Z(8) @ Y(9))
  + (-1.043994808861544e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ X(7) @ Z(8) @ X(9))
  + (-1.043994808861544e-06+0j) * (X(1) @ Z(2) @ X(3) @ Y(7) @ Z(8) @ Y(9))
  + (-1.1514041160251484e-06+0j) * (X(1) @ Z(2) @ X(3) @ X(7) @ Z(8) @ X(9))
  + (-1.0740930716360443e-07+0j) * (X(1) @ Z(2) @ Y(3) @ Y(7) @ Z(8) @ X(9))
  + (-1.9921685508420358e-07+0j) * (Y(1) @ Z(2) @ X(3) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-7.482906647794253e-07+0j) * (Y(1) @ Z(2) @ Y(3) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-5.490738096952216e-07+0j) * (Y(1) @ Z(2) @ Y(3) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-5.490738096952216e-07+0j) * (X(1) @ Z(2) @ X(3) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-7.482906647794253e-07+0j) * (X(1) @ Z(2) @ X(3) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.9921685508420358e-07+0j) * (X(1) @ Z(2) @ Y(3) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (2.57261277471324e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Z(9))
  + (2.57261277471324e-06+0j) * (X(1) @ Z(2) @ X(3) @ Z(9))
  + (-5.8946036810601414e-08+0j) * (Y(1) @ Z(2) @ X(3) @ X(9) @ Z(10) @ Y(11))
  + (-1.4084316502240574e-07+0j) * (Y(1) @ Z(2) @ Y(3) @ Y(9) @ Z(10) @ Y(11))
  + (-8.189712821180429e-08+0j) * (Y(1) @ Z(2) @ Y(3) @ X(9) @ Z(10) @ X(11))
  + (-8.189712821180429e-08+0j) * (X(1) @ Z(2) @ X(3) @ Y(9) @ Z(10) @ Y(11))
  + (-1.4084316502240574e-07+0j) * (X(1) @ Z(2) @ X(3) @ X(9) @ Z(10) @ X(11))
  + (-5.8946036810601414e-08+0j) * (X(1) @ Z(2) @ Y(3) @ Y(9) @ Z(10) @ X(11))
  + (8.683108875713097e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Z(11))
  + (8.683108875713097e-06+0j) * (X(1) @ Z(2) @ X(3) @ Z(11))
  + (0.10654990999225297+0j) * (Z(1) @ Z(4))
  + (8.544528209807287e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Z(4))
  + (8.544528209807287e-06+0j) * (X(1) @ Z(2) @ X(3) @ Z(4))
  + (-1.3842259855013354e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(5))
  + (-1.3842259855013354e-05+0j) * (X(1) @ Z(2) @ Z(3) @ X(5))
  + (-0.053299979303300576+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Y(7))
  + (-0.053299979303300576+0j) * (X(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ X(7))
  + (-0.0031957805885567793+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-0.0031957805885567793+0j) * (X(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (0.0021394350862313644+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.0021394350862313644+0j) * (X(1) @ Z(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (1.112644010242348e-06+0j) * (Z(1) @ Y(4) @ Z(5) @ Y(6))
  + (1.112644010242348e-06+0j) * (Z(1) @ X(4) @ Z(5) @ X(6))
  + (-9.444518897101256e-07+0j) * (Y(1) @ Y(4) @ Z(5) @ Y(6) @ Z(2) @ Y(3))
  + (-9.444518897101256e-07+0j) * (Y(1) @ X(4) @ Z(5) @ X(6) @ Z(2) @ Y(3))
  + (-9.444518897101256e-07+0j) * (X(1) @ Y(4) @ Z(5) @ Y(6) @ Z(2) @ X(3))
  + (-9.444518897101256e-07+0j) * (X(1) @ X(4) @ Z(5) @ X(6) @ Z(2) @ X(3))
  + (0.0018542767130606034+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(5) @ Y(6))
  + (-0.0018542767130606034+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ X(5) @ X(6))
  + (-0.0018542767130606034+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ Y(5) @ Y(6))
  + (0.0018542767130606034+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(5) @ X(6))
  + (4.989857748451018e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Y(7))
  + (4.989857748451018e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Y(7))
  + (4.989857748451018e-06+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ X(7))
  + (4.989857748451018e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ X(7))
  + (-1.1665999509906146e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Y(9))
  + (-1.1665999509906146e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Y(9))
  + (-1.1665999509906146e-06+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ X(9))
  + (-1.1665999509906146e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ X(9))
  + (-2.762169875580479e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-2.762169875580479e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-2.762169875580479e-07+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-2.762169875580479e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (2.89923381181582e-06+0j) * (Z(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (2.89923381181582e-06+0j) * (Z(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-2.93225664649567e-07+0j) * (Y(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(2) @ Y(3))
  + (-2.93225664649567e-07+0j) * (Y(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(2) @ Y(3))
  + (-2.93225664649567e-07+0j) * (X(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(2) @ X(3))
  + (-2.93225664649567e-07+0j) * (X(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(2) @ X(3))
  + (-0.00017591273911803616+0j) * (Y(1) @ X(4) @ X(5) @ Z(6) @ Z(7) @ Y(8) @ Z(2) @ Z(3))
  + (0.00017591273911803616+0j) * (Y(1) @ Y(4) @ X(5) @ Z(6) @ Z(7) @ X(8) @ Z(2) @ Z(3))
  + (0.00017591273911803616+0j) * (X(1) @ X(4) @ Y(5) @ Z(6) @ Z(7) @ Y(8) @ Z(2) @ Z(3))
  + (-0.00017591273911803616+0j) * (X(1) @ Y(4) @ Y(5) @ Z(6) @ Z(7) @ X(8) @ Z(2) @ Z(3))
  + (3.5959028358316347e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(7) @ Y(8))
  + (-3.5959028358316347e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ X(7) @ X(8))
  + (-3.5959028358316347e-07+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ Y(7) @ Y(8))
  + (3.5959028358316347e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(7) @ X(8))
  + (1.6081416347229119e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(8) @ Y(9))
  + (1.6081416347229119e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ Y(8) @ Y(9))
  + (1.6081416347229119e-06+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ X(8) @ X(9))
  + (1.6081416347229119e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(8) @ X(9))
  + (-6.683321744608681e-08+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (-6.683321744608681e-08+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (-6.683321744608681e-08+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (-6.683321744608681e-08+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (-4.763021047372312e-08+0j) * (Z(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-4.763021047372312e-08+0j) * (Z(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.00025082396844776367+0j) * (Y(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ Y(3))
  + (0.00025082396844776367+0j) * (Y(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ Y(3))
  + (0.00025082396844776367+0j) * (X(1) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ X(3))
  + (0.00025082396844776367+0j) * (X(1) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ X(3))
  + (0.0008108121025902371+0j) * (Y(1) @ X(4) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ Z(3))
  + (-0.0008108121025902371+0j) * (Y(1) @ Y(4) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ Z(3))
  + (-0.0008108121025902371+0j) * (X(1) @ X(4) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ Z(3))
  + (0.0008108121025902371+0j) * (X(1) @ Y(4) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ Z(3))
  + (1.407430566218431e-07+0j) * (Y(1) @ X(4) @ X(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ Z(3))
  + (-1.407430566218431e-07+0j) * (Y(1) @ Y(4) @ X(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ Z(3))
  + (-1.407430566218431e-07+0j) * (X(1) @ X(4) @ Y(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ Z(3))
  + (1.407430566218431e-07+0j) * (X(1) @ Y(4) @ Y(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ Z(3))
  + (-3.404142811488842e-08+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(9) @ Y(10))
  + (3.404142811488842e-08+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ X(9) @ X(10))
  + (3.404142811488842e-08+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ Y(9) @ Y(10))
  + (-3.404142811488842e-08+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(9) @ X(10))
  + (-7.347550678322494e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ X(4) @ X(10) @ Y(11))
  + (-7.347550678322494e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Y(4) @ Y(10) @ Y(11))
  + (-7.347550678322494e-07+0j) * (X(1) @ Z(2) @ Z(3) @ X(4) @ X(10) @ X(11))
  + (-7.347550678322494e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Y(4) @ Y(10) @ X(11))
  + (-0.05144570259023998+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Y(7))
  + (-0.05144570259023998+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ X(7))
  + (-0.0033716933276748155+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-0.0033716933276748155+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (0.002950247188821601+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.002950247188821601+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.3853960588383758e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(7))
  + (-1.3853960588383758e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(7))
  + (-8.070096674074511e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Y(9))
  + (1.5368154325893712e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Y(9))
  + (2.3438250999968223e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(7) @ Z(8) @ X(9))
  + (2.3438250999968223e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(7) @ Z(8) @ Y(9))
  + (1.5368154325893712e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ X(9))
  + (-8.070096674074511e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ X(9))
  + (-1.3547393093620481e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-6.09663819969118e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-4.7418988903291327e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-4.7418988903291327e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-6.09663819969118e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.3547393093620481e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-3.9898308887889304e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(9))
  + (-3.9898308887889304e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(9))
  + (-1.0087464556097522e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(9) @ Z(10) @ Y(11))
  + (-1.369110775400953e-07+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(9) @ Z(10) @ Y(11))
  + (-3.603643197912009e-08+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(9) @ Z(10) @ X(11))
  + (-3.603643197912009e-08+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(9) @ Z(10) @ Y(11))
  + (-1.369110775400953e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(9) @ Z(10) @ X(11))
  + (-1.0087464556097522e-07+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(9) @ Z(10) @ X(11))
  + (-1.2599081894237662e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(11))
  + (-1.2599081894237662e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(11))
  + (0.10838382536484995+0j) * (Z(1) @ Z(6))
  + (6.1018797452128895e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Z(6))
  + (6.1018797452128895e-06+0j) * (X(1) @ Z(2) @ X(3) @ Z(6))
  + (-8.86410283993274e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(6))
  + (-8.86410283993274e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(6))
  + (-0.03934646448990639+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(7))
  + (-0.03934646448990639+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(7))
  + (-0.009537445350450844+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Y(9))
  + (-0.009537445350450844+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ X(9))
  + (4.034512916619667e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (4.034512916619667e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.0017591729595976973+0j) * (Z(1) @ Y(6) @ Z(7) @ Y(8))
  + (0.0017591729595976973+0j) * (Z(1) @ X(6) @ Z(7) @ X(8))
  + (-2.0344278062959845e-06+0j) * (Y(1) @ Y(6) @ Z(7) @ Y(8) @ Z(2) @ Y(3))
  + (-2.0344278062959845e-06+0j) * (Y(1) @ X(6) @ Z(7) @ X(8) @ Z(2) @ Y(3))
  + (-2.0344278062959845e-06+0j) * (X(1) @ Y(6) @ Z(7) @ Y(8) @ Z(2) @ X(3))
  + (-2.0344278062959845e-06+0j) * (X(1) @ X(6) @ Z(7) @ X(8) @ Z(2) @ X(3))
  + (2.7034153835799853e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(6) @ Z(7) @ Y(8))
  + (2.7034153835799853e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(6) @ Z(7) @ X(8))
  + (2.7034153835799853e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(6) @ Z(7) @ Y(8))
  + (2.7034153835799853e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(6) @ Z(7) @ X(8))
  + (0.013264685358089088+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(7) @ Y(8))
  + (-0.013264685358089088+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(7) @ X(8))
  + (-0.013264685358089088+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(7) @ Y(8))
  + (0.013264685358089088+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(7) @ X(8))
  + (0.004561593662947706+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Y(9))
  + (0.004561593662947706+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Y(9))
  + (0.004561593662947706+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ X(9))
  + (0.004561593662947706+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ X(9))
  + (1.2337859629559085e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.2337859629559085e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.2337859629559085e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (1.2337859629559085e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.1318270471508736e-07+0j) * (Z(1) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-1.1318270471508736e-07+0j) * (Z(1) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-4.112770860654544e-07+0j) * (Y(1) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ Y(3))
  + (-4.112770860654544e-07+0j) * (Y(1) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ Y(3))
  + (-4.112770860654544e-07+0j) * (X(1) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ X(3))
  + (-4.112770860654544e-07+0j) * (X(1) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ X(3))
  + (-3.334468324110703e-07+0j) * (Y(1) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ Z(3) @ Z(4) @ Y(5))
  + (-3.334468324110703e-07+0j) * (Y(1) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ Z(3) @ Z(4) @ Y(5))
  + (-3.334468324110703e-07+0j) * (X(1) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(2) @ Z(3) @ Z(4) @ X(5))
  + (-3.334468324110703e-07+0j) * (X(1) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(2) @ Z(3) @ Z(4) @ X(5))
  + (1.6544689860071028e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (-1.6544689860071028e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.6544689860071028e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (1.6544689860071028e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (-4.109859674805216e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(9) @ Y(10))
  + (4.109859674805216e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(9) @ X(10))
  + (4.109859674805216e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(9) @ Y(10))
  + (-4.109859674805216e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(9) @ X(10))
  + (-0.003977948028580935+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(10) @ Y(11))
  + (-0.003977948028580935+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(10) @ Y(11))
  + (-0.003977948028580935+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(10) @ X(11))
  + (-0.003977948028580935+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(10) @ X(11))
  + (0.0037272400076382452+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Y(9))
  + (0.0037272400076382452+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ X(9))
  + (5.68898190262677e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (5.68898190262677e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.01508988367698451+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(9))
  + (-0.01508988367698451+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(9))
  + (8.22799995475387e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ Z(10) @ Y(11))
  + (1.1118289109801853e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ Z(10) @ Y(11))
  + (2.890289155047985e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(9) @ Z(10) @ X(11))
  + (2.890289155047985e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(9) @ Z(10) @ Y(11))
  + (1.1118289109801853e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ Z(10) @ X(11))
  + (8.22799995475387e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ Z(10) @ X(11))
  + (-0.050866803551057625+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(11))
  + (-0.050866803551057625+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(11))
  + (0.06670683844071446+0j) * (Z(1) @ Z(8))
  + (1.792100095493261e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Z(8))
  + (1.792100095493261e-06+0j) * (X(1) @ Z(2) @ X(3) @ Z(8))
  + (-2.3816892540660184e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(8))
  + (-2.3816892540660184e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(8))
  + (-0.010528290014036804+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(8))
  + (-0.010528290014036804+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(8))
  + (0.0008869981212886137+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(9))
  + (0.0008869981212886137+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(9))
  + (1.1617641017697768e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ Z(10) @ Y(11))
  + (1.1617641017697768e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ Z(10) @ X(11))
  + (-1.2642530177365674e-05+0j) * (Z(1) @ Y(8) @ Z(9) @ Y(10))
  + (-1.2642530177365674e-05+0j) * (Z(1) @ X(8) @ Z(9) @ X(10))
  + (-3.958585014951183e-08+0j) * (Y(1) @ Y(8) @ Z(9) @ Y(10) @ Z(2) @ Y(3))
  + (-3.958585014951183e-08+0j) * (Y(1) @ X(8) @ Z(9) @ X(10) @ Z(2) @ Y(3))
  + (-3.958585014951183e-08+0j) * (X(1) @ Y(8) @ Z(9) @ Y(10) @ Z(2) @ X(3))
  + (-3.958585014951183e-08+0j) * (X(1) @ X(8) @ Z(9) @ X(10) @ Z(2) @ X(3))
  + (-7.007786009400852e-08+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Y(8) @ Z(9) @ Y(10))
  + (-7.007786009400852e-08+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ X(8) @ Z(9) @ X(10))
  + (-7.007786009400852e-08+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Y(8) @ Z(9) @ Y(10))
  + (-7.007786009400852e-08+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ X(8) @ Z(9) @ X(10))
  + (-1.2195705197572306e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(8) @ Z(9) @ Y(10))
  + (-1.2195705197572306e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(8) @ Z(9) @ X(10))
  + (-1.2195705197572306e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(8) @ Z(9) @ Y(10))
  + (-1.2195705197572306e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(8) @ Z(9) @ X(10))
  + (5.852520098764472e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(9) @ Y(10))
  + (-5.852520098764472e-06+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ X(9) @ X(10))
  + (-5.852520098764472e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Y(9) @ Y(10))
  + (5.852520098764472e-06+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(9) @ X(10))
  + (0.0003207745359311322+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(10) @ Y(11))
  + (0.0003207745359311322+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(10) @ Y(11))
  + (0.0003207745359311322+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(10) @ X(11))
  + (0.0003207745359311322+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(10) @ X(11))
  + (1.747016111646224e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(10) @ Y(11))
  + (1.747016111646224e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(10) @ X(11))
  + (-0.0011212419840400235+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(11))
  + (-0.0011212419840400235+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(11))
  + (0.10547204009808439+0j) * (Z(1) @ Z(10))
  + (9.235851457594781e-06+0j) * (Y(1) @ Z(2) @ Y(3) @ Z(10))
  + (9.235851457594781e-06+0j) * (X(1) @ Z(2) @ X(3) @ Z(10))
  + (-1.3333836962069914e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Y(5) @ Z(10))
  + (-1.3333836962069914e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ X(5) @ Z(10))
  + (-0.054844751579638557+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(10))
  + (-0.054844751579638557+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(10))
  + (-0.000800467448108891+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(10))
  + (-0.000800467448108891+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(10))
  + (7.286853052733687e-05+0j) * (Y(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(11))
  + (7.286853052733687e-05+0j) * (X(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(11))
  + (0.1806398639109381+0j) * (Z(2) @ Z(4))
  + (8.352895074206844e-06+0j) * (Z(2) @ Y(4) @ Z(5) @ Y(6))
  + (8.352895074206844e-06+0j) * (Z(2) @ X(4) @ Z(5) @ X(6))
  + (2.60163635543075e-06+0j) * (Z(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (2.60163635543075e-06+0j) * (Z(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-6.066041301222396e-07+0j) * (Z(2) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-6.066041301222396e-07+0j) * (Z(2) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.16273984247398934+0j) * (Z(2) @ Z(6))
  + (-0.016759934370256715+0j) * (Z(2) @ Y(6) @ Z(7) @ Y(8))
  + (-0.016759934370256715+0j) * (Z(2) @ X(6) @ Z(7) @ X(8))
  + (-0.005136935925690359+0j) * (Z(2) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.005136935925690359+0j) * (Z(2) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.08190949436414979+0j) * (Z(2) @ Z(8))
  + (-0.0009856117931238539+0j) * (Z(2) @ Y(8) @ Z(9) @ Y(10))
  + (-0.0009856117931238539+0j) * (Z(2) @ X(8) @ Z(9) @ X(10))
  + (0.18723114719973982+0j) * (Z(2) @ Z(10))
  + (0.20347771138739176+0j) * (Z(2) @ Z(3))
  + (6.182834567982184e-07+0j) * (Y(2) @ Y(4))
  + (6.182834567982184e-07+0j) * (X(2) @ X(4))
  + (-5.302559049597934e-06+0j) * (Y(2) @ Z(4) @ Z(5) @ Y(6))
  + (-5.302559049597934e-06+0j) * (X(2) @ Z(4) @ Z(5) @ X(6))
  + (-4.3981578678506605e-08+0j) * (Y(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-4.3981578678506605e-08+0j) * (X(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-2.3178887244666677e-08+0j) * (Y(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-2.3178887244666677e-08+0j) * (X(2) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (6.182834567982184e-07+0j) * (Z(2) @ Y(3) @ Z(4) @ Y(5))
  + (6.182834567982184e-07+0j) * (Z(2) @ X(3) @ Z(4) @ X(5))
  + (0.007612747733046051+0j) * (Y(2) @ X(3) @ X(4) @ Y(5))
  + (-0.007612747733046051+0j) * (Y(2) @ Y(3) @ X(4) @ X(5))
  + (-0.007612747733046051+0j) * (X(2) @ X(3) @ Y(4) @ Y(5))
  + (0.007612747733046051+0j) * (X(2) @ Y(3) @ Y(4) @ X(5))
  + (5.666793526845813e-07+0j) * (Y(2) @ X(3) @ X(5) @ Y(6))
  + (5.666793526845813e-07+0j) * (Y(2) @ Y(3) @ Y(5) @ Y(6))
  + (5.666793526845813e-07+0j) * (X(2) @ X(3) @ X(5) @ X(6))
  + (5.666793526845813e-07+0j) * (X(2) @ Y(3) @ Y(5) @ X(6))
  + (3.80060247132713e-08+0j) * (Y(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Y(8))
  + (3.80060247132713e-08+0j) * (Y(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Y(8))
  + (3.80060247132713e-08+0j) * (X(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ X(8))
  + (3.80060247132713e-08+0j) * (X(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ X(8))
  + (3.0029499313424975e-07+0j) * (Y(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (3.0029499313424975e-07+0j) * (Y(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (3.0029499313424975e-07+0j) * (X(2) @ X(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (3.0029499313424975e-07+0j) * (X(2) @ Y(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-5.302559049597934e-06+0j) * (Z(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7))
  + (-5.302559049597934e-06+0j) * (Z(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7))
  + (-5.666793526845812e-07+0j) * (Y(2) @ X(3) @ X(4) @ Z(5) @ Z(6) @ Y(7))
  + (5.666793526845812e-07+0j) * (Y(2) @ Y(3) @ X(4) @ Z(5) @ Z(6) @ X(7))
  + (5.666793526845812e-07+0j) * (X(2) @ X(3) @ Y(4) @ Z(5) @ Z(6) @ Y(7))
  + (-5.666793526845812e-07+0j) * (X(2) @ Y(3) @ Y(4) @ Z(5) @ Z(6) @ X(7))
  + (0.004505868156451894+0j) * (Y(2) @ X(3) @ X(6) @ Y(7))
  + (-0.004505868156451894+0j) * (Y(2) @ Y(3) @ X(6) @ X(7))
  + (-0.004505868156451894+0j) * (X(2) @ X(3) @ Y(6) @ Y(7))
  + (0.004505868156451894+0j) * (X(2) @ Y(3) @ Y(6) @ X(7))
  + (-0.0009354682839745884+0j) * (Y(2) @ X(3) @ X(7) @ Y(8))
  + (-0.0009354682839745884+0j) * (Y(2) @ Y(3) @ Y(7) @ Y(8))
  + (-0.0009354682839745884+0j) * (X(2) @ X(3) @ X(7) @ X(8))
  + (-0.0009354682839745884+0j) * (X(2) @ Y(3) @ Y(7) @ X(8))
  + (0.0017270185275798833+0j) * (Y(2) @ X(3) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.0017270185275798833+0j) * (Y(2) @ Y(3) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.0017270185275798833+0j) * (X(2) @ X(3) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (0.0017270185275798833+0j) * (X(2) @ Y(3) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (-4.3981578678506605e-08+0j) * (Z(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-4.3981578678506605e-08+0j) * (Z(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-3.80060247132713e-08+0j) * (Y(2) @ X(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (3.80060247132713e-08+0j) * (Y(2) @ Y(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (3.80060247132713e-08+0j) * (X(2) @ X(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-3.80060247132713e-08+0j) * (X(2) @ Y(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (0.0009354682839745884+0j) * (Y(2) @ X(3) @ X(6) @ Z(7) @ Z(8) @ Y(9))
  + (-0.0009354682839745884+0j) * (Y(2) @ Y(3) @ X(6) @ Z(7) @ Z(8) @ X(9))
  + (-0.0009354682839745884+0j) * (X(2) @ X(3) @ Y(6) @ Z(7) @ Z(8) @ Y(9))
  + (0.0009354682839745884+0j) * (X(2) @ Y(3) @ Y(6) @ Z(7) @ Z(8) @ X(9))
  + (0.002359359535679982+0j) * (Y(2) @ X(3) @ X(8) @ Y(9))
  + (-0.002359359535679982+0j) * (Y(2) @ Y(3) @ X(8) @ X(9))
  + (-0.002359359535679982+0j) * (X(2) @ X(3) @ Y(8) @ Y(9))
  + (0.002359359535679982+0j) * (X(2) @ Y(3) @ Y(8) @ X(9))
  + (0.0008306757992544091+0j) * (Y(2) @ X(3) @ X(9) @ Y(10))
  + (0.0008306757992544091+0j) * (Y(2) @ Y(3) @ Y(9) @ Y(10))
  + (0.0008306757992544091+0j) * (X(2) @ X(3) @ X(9) @ X(10))
  + (0.0008306757992544091+0j) * (X(2) @ Y(3) @ Y(9) @ X(10))
  + (-2.317888724466668e-08+0j) * (Z(2) @ Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-2.317888724466668e-08+0j) * (Z(2) @ X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-3.002949931342498e-07+0j) * (Y(2) @ X(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (3.002949931342498e-07+0j) * (Y(2) @ Y(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (3.002949931342498e-07+0j) * (X(2) @ X(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-3.002949931342498e-07+0j) * (X(2) @ Y(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.0017270185275798833+0j) * (Y(2) @ X(3) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.0017270185275798833+0j) * (Y(2) @ Y(3) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.0017270185275798833+0j) * (X(2) @ X(3) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0017270185275798833+0j) * (X(2) @ Y(3) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.0008306757992544091+0j) * (Y(2) @ X(3) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.0008306757992544091+0j) * (Y(2) @ Y(3) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (0.0008306757992544091+0j) * (X(2) @ X(3) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0008306757992544091+0j) * (X(2) @ Y(3) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (0.007950935708453516+0j) * (Y(2) @ X(3) @ X(10) @ Y(11))
  + (-0.007950935708453516+0j) * (Y(2) @ Y(3) @ X(10) @ X(11))
  + (-0.007950935708453516+0j) * (X(2) @ X(3) @ Y(10) @ Y(11))
  + (0.007950935708453516+0j) * (X(2) @ Y(3) @ Y(10) @ X(11))
  + (-4.581409691916594e-06+0j) * (Y(2) @ Z(3) @ Z(5) @ Y(6))
  + (-4.581409691916594e-06+0j) * (X(2) @ Z(3) @ Z(5) @ X(6))
  + (-1.0848645138317281e-07+0j) * (Y(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (-1.0848645138317281e-07+0j) * (X(2) @ Z(3) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-3.4639936372740286e-06+0j) * (Y(2) @ Z(3) @ Y(4) @ Z(6))
  + (-3.4639936372740286e-06+0j) * (X(2) @ Z(3) @ X(4) @ Z(6))
  + (2.754157082292255e-10+0j) * (Y(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Y(8))
  + (2.2177618557614032e-07+0j) * (Y(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Y(8))
  + (2.2150076986791115e-07+0j) * (Y(2) @ Z(3) @ Y(4) @ X(6) @ Z(7) @ X(8))
  + (2.2150076986791115e-07+0j) * (X(2) @ Z(3) @ X(4) @ Y(6) @ Z(7) @ Y(8))
  + (2.2177618557614032e-07+0j) * (X(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ X(8))
  + (2.754157082292255e-10+0j) * (X(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ X(8))
  + (-1.8629523159388112e-06+0j) * (Y(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0015719412938846463+0j) * (Y(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0015700783415687081+0j) * (Y(2) @ Z(3) @ Y(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-0.0015700783415687081+0j) * (X(2) @ Z(3) @ X(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0015719412938846463+0j) * (X(2) @ Z(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.8629523159388112e-06+0j) * (X(2) @ Z(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (1.4872849500815375e-07+0j) * (Y(2) @ Z(3) @ Y(4) @ Z(8))
  + (1.4872849500815375e-07+0j) * (X(2) @ Z(3) @ X(4) @ Z(8))
  + (4.5280933431456126e-09+0j) * (Y(2) @ Z(3) @ X(4) @ X(8) @ Z(9) @ Y(10))
  + (-0.00029972293184105244+0j) * (Y(2) @ Z(3) @ Y(4) @ Y(8) @ Z(9) @ Y(10))
  + (-0.00029972745993439554+0j) * (Y(2) @ Z(3) @ Y(4) @ X(8) @ Z(9) @ X(10))
  + (-0.00029972745993439554+0j) * (X(2) @ Z(3) @ X(4) @ Y(8) @ Z(9) @ Y(10))
  + (-0.00029972293184105244+0j) * (X(2) @ Z(3) @ X(4) @ X(8) @ Z(9) @ X(10))
  + (4.5280933431456126e-09+0j) * (X(2) @ Z(3) @ Y(4) @ Y(8) @ Z(9) @ X(10))
  + (4.431736140047339e-08+0j) * (Y(2) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(3))
  + (4.431736140047339e-08+0j) * (X(2) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(3))
  + (3.38313126165121e-06+0j) * (Y(2) @ Z(3) @ Y(4) @ Z(10))
  + (3.38313126165121e-06+0j) * (X(2) @ Z(3) @ X(4) @ Z(10))
  + (0.18825261164398413+0j) * (Z(2) @ Z(5))
  + (6.182859268279962e-07+0j) * (Y(2) @ Z(3) @ Y(4) @ Z(5))
  + (6.182859268279962e-07+0j) * (X(2) @ Z(3) @ X(4) @ Z(5))
  + (-4.821187120733454e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Y(6))
  + (-4.821187120733454e-06+0j) * (X(2) @ Z(3) @ Z(4) @ X(6))
  + (-7.563187351676158e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ Y(8))
  + (-7.563187351676158e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(6) @ Z(7) @ X(8))
  + (8.919574426891426e-06+0j) * (Z(2) @ Y(5) @ Z(6) @ Y(7))
  + (8.919574426891426e-06+0j) * (Z(2) @ X(5) @ Z(6) @ X(7))
  + (2.397774288168595e-07+0j) * (Y(2) @ Y(5) @ Z(6) @ Y(7) @ Z(3) @ Y(4))
  + (2.397774288168595e-07+0j) * (Y(2) @ X(5) @ Z(6) @ X(7) @ Z(3) @ Y(4))
  + (2.397774288168595e-07+0j) * (X(2) @ Y(5) @ Z(6) @ Y(7) @ Z(3) @ X(4))
  + (2.397774288168595e-07+0j) * (X(2) @ X(5) @ Z(6) @ X(7) @ Z(3) @ X(4))
  + (-1.326442165621555e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ X(5) @ X(6) @ Y(7))
  + (1.326442165621555e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Y(5) @ X(6) @ X(7))
  + (1.326442165621555e-06+0j) * (X(2) @ Z(3) @ Z(4) @ X(5) @ Y(6) @ Y(7))
  + (-1.326442165621555e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Y(5) @ Y(6) @ X(7))
  + (3.0603094262736165e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Y(8))
  + (3.0603094262736165e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Y(8))
  + (3.0603094262736165e-07+0j) * (X(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ X(8))
  + (3.0603094262736165e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ X(8))
  + (-0.0005329753877048543+0j) * (Y(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0005329753877048543+0j) * (Y(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0005329753877048543+0j) * (X(2) @ Z(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (-0.0005329753877048543+0j) * (X(2) @ Z(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (2.639642380144021e-06+0j) * (Z(2) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (2.639642380144021e-06+0j) * (Z(2) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-3.285457786641123e-08+0j) * (Y(2) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(3) @ Y(4))
  + (-3.285457786641123e-08+0j) * (Y(2) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(3) @ Y(4))
  + (-3.285457786641123e-08+0j) * (X(2) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(3) @ X(4))
  + (-3.285457786641123e-08+0j) * (X(2) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(3) @ X(4))
  + (-3.057555269191325e-07+0j) * (Y(2) @ X(5) @ X(6) @ Z(7) @ Z(8) @ Y(9) @ Z(3) @ Z(4))
  + (3.057555269191325e-07+0j) * (Y(2) @ Y(5) @ X(6) @ Z(7) @ Z(8) @ X(9) @ Z(3) @ Z(4))
  + (3.057555269191325e-07+0j) * (X(2) @ X(5) @ Y(6) @ Z(7) @ Z(8) @ Y(9) @ Z(3) @ Z(4))
  + (-3.057555269191325e-07+0j) * (X(2) @ Y(5) @ Y(6) @ Z(7) @ Z(8) @ X(9) @ Z(3) @ Z(4))
  + (3.131740266717324e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ X(5) @ X(8) @ Y(9))
  + (-3.131740266717324e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Y(5) @ X(8) @ X(9))
  + (-3.131740266717324e-08+0j) * (X(2) @ Z(3) @ Z(4) @ X(5) @ Y(8) @ Y(9))
  + (3.131740266717324e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Y(5) @ Y(8) @ X(9))
  + (-0.00025596915884023804+0j) * (Y(2) @ Z(3) @ Z(4) @ X(5) @ X(9) @ Y(10))
  + (-0.00025596915884023804+0j) * (Y(2) @ Z(3) @ Z(4) @ Y(5) @ Y(9) @ Y(10))
  + (-0.00025596915884023804+0j) * (X(2) @ Z(3) @ Z(4) @ X(5) @ X(9) @ X(10))
  + (-0.00025596915884023804+0j) * (X(2) @ Z(3) @ Z(4) @ Y(5) @ Y(9) @ X(10))
  + (-3.0630913698798975e-07+0j) * (Z(2) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-3.0630913698798975e-07+0j) * (Z(2) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (4.431736140047339e-08+0j) * (Y(2) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ Y(4))
  + (4.431736140047339e-08+0j) * (Y(2) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ Y(4))
  + (4.431736140047339e-08+0j) * (X(2) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ X(4))
  + (4.431736140047339e-08+0j) * (X(2) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ X(4))
  + (0.0005311124353889156+0j) * (Y(2) @ X(5) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ Z(4))
  + (-0.0005311124353889156+0j) * (Y(2) @ Y(5) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ Z(4))
  + (-0.0005311124353889156+0j) * (X(2) @ X(5) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ Z(4))
  + (0.0005311124353889156+0j) * (X(2) @ Y(5) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ Z(4))
  + (0.0002559736869335812+0j) * (Y(2) @ X(5) @ X(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ Z(4))
  + (-0.0002559736869335812+0j) * (Y(2) @ Y(5) @ X(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ Z(4))
  + (-0.0002559736869335812+0j) * (X(2) @ X(5) @ Y(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ Z(4))
  + (0.0002559736869335812+0j) * (X(2) @ Y(5) @ Y(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ Z(4))
  + (8.200515073377094e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ X(5) @ X(10) @ Y(11))
  + (-8.200515073377094e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Y(5) @ X(10) @ X(11))
  + (-8.200515073377094e-07+0j) * (X(2) @ Z(3) @ Z(4) @ X(5) @ Y(10) @ Y(11))
  + (8.200515073377094e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Y(5) @ Y(10) @ X(11))
  + (4.776953729518488e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Y(8))
  + (4.776953729518488e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ X(8))
  + (3.9790052479638824e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (3.9790052479638824e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-1.4481289214215305e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(8))
  + (-1.4481289214215305e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(8))
  + (8.194466821363216e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ Y(10))
  + (5.367507516707476e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ Y(10))
  + (-2.82695930465574e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(8) @ Z(9) @ X(10))
  + (-2.82695930465574e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(8) @ Z(9) @ Y(10))
  + (5.367507516707476e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ X(10))
  + (8.194466821363216e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ X(10))
  + (-4.9338304715739734e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(10))
  + (-4.9338304715739734e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(10))
  + (0.16724571063044122+0j) * (Z(2) @ Z(7))
  + (-2.137551471652473e-06+0j) * (Y(2) @ Z(3) @ Y(4) @ Z(7))
  + (-2.137551471652473e-06+0j) * (X(2) @ Z(3) @ X(4) @ Z(7))
  + (-3.6243092517772764e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(7))
  + (-3.6243092517772764e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(7))
  + (-8.068133376932066e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(8))
  + (-8.068133376932066e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(8))
  + (2.832227281896245e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ Y(10))
  + (2.832227281896245e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ X(10))
  + (-0.017695402654231304+0j) * (Z(2) @ Y(7) @ Z(8) @ Y(9))
  + (-0.017695402654231304+0j) * (Z(2) @ X(7) @ Z(8) @ X(9))
  + (-8.425475705122133e-08+0j) * (Y(2) @ Y(7) @ Z(8) @ Y(9) @ Z(3) @ Y(4))
  + (-8.425475705122133e-08+0j) * (Y(2) @ X(7) @ Z(8) @ X(9) @ Z(3) @ Y(4))
  + (-8.425475705122133e-08+0j) * (X(2) @ Y(7) @ Z(8) @ Y(9) @ Z(3) @ X(4))
  + (-8.425475705122133e-08+0j) * (X(2) @ X(7) @ Z(8) @ X(9) @ Z(3) @ X(4))
  + (1.2845087106450552e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(7) @ Z(8) @ Y(9))
  + (1.2845087106450552e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(7) @ Z(8) @ X(9))
  + (1.2845087106450552e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(7) @ Z(8) @ Y(9))
  + (1.2845087106450552e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(7) @ Z(8) @ X(9))
  + (-4.971119872092056e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(8) @ Y(9))
  + (4.971119872092056e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(8) @ X(9))
  + (4.971119872092056e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(8) @ Y(9))
  + (-4.971119872092056e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(8) @ X(9))
  + (6.987412771903144e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ Y(10))
  + (6.987412771903144e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ Y(10))
  + (6.987412771903144e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ X(10))
  + (6.987412771903144e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ X(10))
  + (-0.0034099173981104753+0j) * (Z(2) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0034099173981104753+0j) * (Z(2) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.0010389659061797922+0j) * (Y(2) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ Y(4))
  + (-0.0010389659061797922+0j) * (Y(2) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ Y(4))
  + (-0.0010389659061797922+0j) * (X(2) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ X(4))
  + (-0.0010389659061797922+0j) * (X(2) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ X(4))
  + (1.1467779660676369e-07+0j) * (Y(2) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (1.1467779660676369e-07+0j) * (Y(2) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ Z(4) @ Z(5) @ Y(6))
  + (1.1467779660676369e-07+0j) * (X(2) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + (1.1467779660676369e-07+0j) * (X(2) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(3) @ Z(4) @ Z(5) @ X(6))
  + (1.2070540494600731e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (-1.2070540494600731e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.2070540494600731e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (1.2070540494600731e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (3.05270754097388e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(10) @ Y(11))
  + (-3.05270754097388e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(10) @ X(11))
  + (-3.05270754097388e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(10) @ Y(11))
  + (3.05270754097388e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(10) @ X(11))
  + (-6.3483818841888935e-09+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ Y(10))
  + (-6.3483818841888935e-09+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ X(10))
  + (4.9382259879352136e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(10))
  + (4.9382259879352136e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(10))
  + (0.08426885389982977+0j) * (Z(2) @ Z(9))
  + (1.1741109234098053e-07+0j) * (Y(2) @ Z(3) @ Y(4) @ Z(9))
  + (1.1741109234098053e-07+0j) * (X(2) @ Z(3) @ X(4) @ Z(9))
  + (-9.51016934212325e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(9))
  + (-9.51016934212325e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(9))
  + (2.1140708919878433e-07+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(9))
  + (2.1140708919878433e-07+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(9))
  + (-4.794572673278472e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(10))
  + (-4.794572673278472e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(10))
  + (-0.00015493599386944468+0j) * (Z(2) @ Y(9) @ Z(10) @ Y(11))
  + (-0.00015493599386944468+0j) * (Z(2) @ X(9) @ Z(10) @ X(11))
  + (-4.3753773000814385e-05+0j) * (Y(2) @ Y(9) @ Z(10) @ Y(11) @ Z(3) @ Y(4))
  + (-4.3753773000814385e-05+0j) * (Y(2) @ X(9) @ Z(10) @ X(11) @ Z(3) @ Y(4))
  + (-4.3753773000814385e-05+0j) * (X(2) @ Y(9) @ Z(10) @ Y(11) @ Z(3) @ X(4))
  + (-4.3753773000814385e-05+0j) * (X(2) @ X(9) @ Z(10) @ X(11) @ Z(3) @ X(4))
  + (-1.6199052551956667e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Y(9) @ Z(10) @ Y(11))
  + (-1.6199052551956667e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ X(9) @ Z(10) @ X(11))
  + (-1.6199052551956667e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Y(9) @ Z(10) @ Y(11))
  + (-1.6199052551956667e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ X(9) @ Z(10) @ X(11))
  + (4.159734484859583e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(9) @ Z(10) @ Y(11))
  + (4.159734484859583e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ X(9) @ Z(10) @ X(11))
  + (4.159734484859583e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Y(9) @ Z(10) @ Y(11))
  + (4.159734484859583e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(9) @ Z(10) @ X(11))
  + (-2.403256387902117e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ X(10) @ Y(11))
  + (2.403256387902117e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ X(10) @ X(11))
  + (2.403256387902117e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Y(10) @ Y(11))
  + (-2.403256387902117e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Y(10) @ X(11))
  + (0.19518208290819333+0j) * (Z(2) @ Z(11))
  + (2.5630797543135006e-06+0j) * (Y(2) @ Z(3) @ Y(4) @ Z(11))
  + (2.5630797543135006e-06+0j) * (X(2) @ Z(3) @ X(4) @ Z(11))
  + (-5.239101225671361e-06+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6) @ Z(11))
  + (-5.239101225671361e-06+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6) @ Z(11))
  + (7.34148237583733e-08+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(11))
  + (7.34148237583733e-08+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(11))
  + (0.1806398639109381+0j) * (Z(3) @ Z(5))
  + (8.352895074206844e-06+0j) * (Z(3) @ Y(5) @ Z(6) @ Y(7))
  + (8.352895074206844e-06+0j) * (Z(3) @ X(5) @ Z(6) @ X(7))
  + (2.60163635543075e-06+0j) * (Z(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (2.60163635543075e-06+0j) * (Z(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-6.066041301222396e-07+0j) * (Z(3) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-6.066041301222396e-07+0j) * (Z(3) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.16273984247398934+0j) * (Z(3) @ Z(7))
  + (-0.016759934370256715+0j) * (Z(3) @ Y(7) @ Z(8) @ Y(9))
  + (-0.016759934370256715+0j) * (Z(3) @ X(7) @ Z(8) @ X(9))
  + (-0.005136935925690359+0j) * (Z(3) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.005136935925690359+0j) * (Z(3) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.08190949436414979+0j) * (Z(3) @ Z(9))
  + (-0.0009856117931238539+0j) * (Z(3) @ Y(9) @ Z(10) @ Y(11))
  + (-0.0009856117931238539+0j) * (Z(3) @ X(9) @ Z(10) @ X(11))
  + (0.18723114719973982+0j) * (Z(3) @ Z(11))
  + (0.18825261164398413+0j) * (Z(3) @ Z(4))
  + (6.182859268279962e-07+0j) * (Y(3) @ Y(5))
  + (6.182859268279962e-07+0j) * (X(3) @ X(5))
  + (-4.821187120733454e-06+0j) * (Y(3) @ Z(5) @ Z(6) @ Y(7))
  + (-4.821187120733454e-06+0j) * (X(3) @ Z(5) @ Z(6) @ X(7))
  + (-7.563187351676158e-08+0j) * (Y(3) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-7.563187351676158e-08+0j) * (X(3) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (8.919574426891426e-06+0j) * (Z(3) @ Y(4) @ Z(5) @ Y(6))
  + (8.919574426891426e-06+0j) * (Z(3) @ X(4) @ Z(5) @ X(6))
  + (2.397774288168595e-07+0j) * (Y(3) @ X(4) @ X(5) @ Y(6))
  + (-2.397774288168595e-07+0j) * (Y(3) @ Y(4) @ X(5) @ X(6))
  + (-2.397774288168595e-07+0j) * (X(3) @ X(4) @ Y(5) @ Y(6))
  + (2.397774288168595e-07+0j) * (X(3) @ Y(4) @ Y(5) @ X(6))
  + (1.3264421656215552e-06+0j) * (Y(3) @ X(4) @ X(6) @ Y(7))
  + (1.3264421656215552e-06+0j) * (Y(3) @ Y(4) @ Y(6) @ Y(7))
  + (1.3264421656215552e-06+0j) * (X(3) @ X(4) @ X(6) @ X(7))
  + (1.3264421656215552e-06+0j) * (X(3) @ Y(4) @ Y(6) @ X(7))
  + (3.0603094262736165e-07+0j) * (Y(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Y(9))
  + (3.0603094262736165e-07+0j) * (Y(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Y(9))
  + (3.0603094262736165e-07+0j) * (X(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ X(9))
  + (3.0603094262736165e-07+0j) * (X(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ X(9))
  + (-0.0005329753877048543+0j) * (Y(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0005329753877048543+0j) * (Y(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0005329753877048543+0j) * (X(3) @ X(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.0005329753877048543+0j) * (X(3) @ Y(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (2.639642380144021e-06+0j) * (Z(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8))
  + (2.639642380144021e-06+0j) * (Z(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8))
  + (-3.285457786641123e-08+0j) * (Y(3) @ X(4) @ X(5) @ Z(6) @ Z(7) @ Y(8))
  + (3.285457786641123e-08+0j) * (Y(3) @ Y(4) @ X(5) @ Z(6) @ Z(7) @ X(8))
  + (3.285457786641123e-08+0j) * (X(3) @ X(4) @ Y(5) @ Z(6) @ Z(7) @ Y(8))
  + (-3.285457786641123e-08+0j) * (X(3) @ Y(4) @ Y(5) @ Z(6) @ Z(7) @ X(8))
  + (-3.057555269191325e-07+0j) * (Y(3) @ X(4) @ X(7) @ Y(8))
  + (3.057555269191325e-07+0j) * (Y(3) @ Y(4) @ X(7) @ X(8))
  + (3.057555269191325e-07+0j) * (X(3) @ X(4) @ Y(7) @ Y(8))
  + (-3.057555269191325e-07+0j) * (X(3) @ Y(4) @ Y(7) @ X(8))
  + (-3.131740266717324e-08+0j) * (Y(3) @ X(4) @ X(8) @ Y(9))
  + (-3.131740266717324e-08+0j) * (Y(3) @ Y(4) @ Y(8) @ Y(9))
  + (-3.131740266717324e-08+0j) * (X(3) @ X(4) @ X(8) @ X(9))
  + (-3.131740266717324e-08+0j) * (X(3) @ Y(4) @ Y(8) @ X(9))
  + (-0.00025596915884023804+0j) * (Y(3) @ X(4) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.00025596915884023804+0j) * (Y(3) @ Y(4) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.00025596915884023804+0j) * (X(3) @ X(4) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.00025596915884023804+0j) * (X(3) @ Y(4) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (-3.0630913698798975e-07+0j) * (Z(3) @ Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-3.0630913698798975e-07+0j) * (Z(3) @ X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (4.431736140047339e-08+0j) * (Y(3) @ X(4) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-4.431736140047339e-08+0j) * (Y(3) @ Y(4) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-4.431736140047339e-08+0j) * (X(3) @ X(4) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (4.431736140047339e-08+0j) * (X(3) @ Y(4) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.0005311124353889156+0j) * (Y(3) @ X(4) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0005311124353889156+0j) * (Y(3) @ Y(4) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (-0.0005311124353889156+0j) * (X(3) @ X(4) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.0005311124353889156+0j) * (X(3) @ Y(4) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (0.0002559736869335812+0j) * (Y(3) @ X(4) @ X(9) @ Y(10))
  + (-0.0002559736869335812+0j) * (Y(3) @ Y(4) @ X(9) @ X(10))
  + (-0.0002559736869335812+0j) * (X(3) @ X(4) @ Y(9) @ Y(10))
  + (0.0002559736869335812+0j) * (X(3) @ Y(4) @ Y(9) @ X(10))
  + (-8.200515073377094e-07+0j) * (Y(3) @ X(4) @ X(10) @ Y(11))
  + (-8.200515073377094e-07+0j) * (Y(3) @ Y(4) @ Y(10) @ Y(11))
  + (-8.200515073377094e-07+0j) * (X(3) @ X(4) @ X(10) @ X(11))
  + (-8.200515073377094e-07+0j) * (X(3) @ Y(4) @ Y(10) @ X(11))
  + (-4.581409691916594e-06+0j) * (Y(3) @ Z(4) @ Z(6) @ Y(7))
  + (-4.581409691916594e-06+0j) * (X(3) @ Z(4) @ Z(6) @ X(7))
  + (-1.0848645138317281e-07+0j) * (Y(3) @ Z(4) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (-1.0848645138317281e-07+0j) * (X(3) @ Z(4) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (-3.4639936372740286e-06+0j) * (Y(3) @ Z(4) @ Y(5) @ Z(7))
  + (-3.4639936372740286e-06+0j) * (X(3) @ Z(4) @ X(5) @ Z(7))
  + (2.754157082292255e-10+0j) * (Y(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Y(9))
  + (2.2177618557614032e-07+0j) * (Y(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Y(9))
  + (2.2150076986791115e-07+0j) * (Y(3) @ Z(4) @ Y(5) @ X(7) @ Z(8) @ X(9))
  + (2.2150076986791115e-07+0j) * (X(3) @ Z(4) @ X(5) @ Y(7) @ Z(8) @ Y(9))
  + (2.2177618557614032e-07+0j) * (X(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ X(9))
  + (2.754157082292255e-10+0j) * (X(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ X(9))
  + (-1.8629523159388112e-06+0j) * (Y(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0015719412938846463+0j) * (Y(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0015700783415687081+0j) * (Y(3) @ Z(4) @ Y(5) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.0015700783415687081+0j) * (X(3) @ Z(4) @ X(5) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0015719412938846463+0j) * (X(3) @ Z(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.8629523159388112e-06+0j) * (X(3) @ Z(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (1.4872849500815375e-07+0j) * (Y(3) @ Z(4) @ Y(5) @ Z(9))
  + (1.4872849500815375e-07+0j) * (X(3) @ Z(4) @ X(5) @ Z(9))
  + (4.5280933431456126e-09+0j) * (Y(3) @ Z(4) @ X(5) @ X(9) @ Z(10) @ Y(11))
  + (-0.00029972293184105244+0j) * (Y(3) @ Z(4) @ Y(5) @ Y(9) @ Z(10) @ Y(11))
  + (-0.00029972745993439554+0j) * (Y(3) @ Z(4) @ Y(5) @ X(9) @ Z(10) @ X(11))
  + (-0.00029972745993439554+0j) * (X(3) @ Z(4) @ X(5) @ Y(9) @ Z(10) @ Y(11))
  + (-0.00029972293184105244+0j) * (X(3) @ Z(4) @ X(5) @ X(9) @ Z(10) @ X(11))
  + (4.5280933431456126e-09+0j) * (X(3) @ Z(4) @ Y(5) @ Y(9) @ Z(10) @ X(11))
  + (4.431736140047339e-08+0j) * (Y(3) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(4))
  + (4.431736140047339e-08+0j) * (X(3) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(4))
  + (3.38313126165121e-06+0j) * (Y(3) @ Z(4) @ Y(5) @ Z(11))
  + (3.38313126165121e-06+0j) * (X(3) @ Z(4) @ X(5) @ Z(11))
  + (0.16724571063044122+0j) * (Z(3) @ Z(6))
  + (-2.137551471652473e-06+0j) * (Y(3) @ Z(4) @ Y(5) @ Z(6))
  + (-2.137551471652473e-06+0j) * (X(3) @ Z(4) @ X(5) @ Z(6))
  + (-3.624309251777276e-06+0j) * (Y(3) @ Z(4) @ Z(5) @ Y(7))
  + (-3.624309251777276e-06+0j) * (X(3) @ Z(4) @ Z(5) @ X(7))
  + (-8.068133376932066e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Y(9))
  + (-8.068133376932066e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ X(9))
  + (2.832227281896245e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (2.832227281896245e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.017695402654231304+0j) * (Z(3) @ Y(6) @ Z(7) @ Y(8))
  + (-0.017695402654231304+0j) * (Z(3) @ X(6) @ Z(7) @ X(8))
  + (-8.425475705122133e-08+0j) * (Y(3) @ Y(6) @ Z(7) @ Y(8) @ Z(4) @ Y(5))
  + (-8.425475705122133e-08+0j) * (Y(3) @ X(6) @ Z(7) @ X(8) @ Z(4) @ Y(5))
  + (-8.425475705122133e-08+0j) * (X(3) @ Y(6) @ Z(7) @ Y(8) @ Z(4) @ X(5))
  + (-8.425475705122133e-08+0j) * (X(3) @ X(6) @ Z(7) @ X(8) @ Z(4) @ X(5))
  + (1.2845087106450552e-06+0j) * (Y(3) @ Z(4) @ Z(5) @ X(6) @ X(7) @ Y(8))
  + (-1.2845087106450552e-06+0j) * (Y(3) @ Z(4) @ Z(5) @ Y(6) @ X(7) @ X(8))
  + (-1.2845087106450552e-06+0j) * (X(3) @ Z(4) @ Z(5) @ X(6) @ Y(7) @ Y(8))
  + (1.2845087106450552e-06+0j) * (X(3) @ Z(4) @ Z(5) @ Y(6) @ Y(7) @ X(8))
  + (4.971119872092056e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Y(9))
  + (4.971119872092056e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Y(9))
  + (4.971119872092056e-07+0j) * (X(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ X(9))
  + (4.971119872092056e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ X(9))
  + (6.987412771903144e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (6.987412771903144e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (6.987412771903144e-08+0j) * (X(3) @ Z(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (6.987412771903144e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.0034099173981104753+0j) * (Z(3) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0034099173981104753+0j) * (Z(3) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (-0.0010389659061797922+0j) * (Y(3) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(4) @ Y(5))
  + (-0.0010389659061797922+0j) * (Y(3) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(4) @ Y(5))
  + (-0.0010389659061797922+0j) * (X(3) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(4) @ X(5))
  + (-0.0010389659061797922+0j) * (X(3) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(4) @ X(5))
  + (1.146777966067637e-07+0j) * (Y(3) @ X(6) @ X(7) @ Z(8) @ Z(9) @ Y(10) @ Z(4) @ Z(5))
  + (-1.146777966067637e-07+0j) * (Y(3) @ Y(6) @ X(7) @ Z(8) @ Z(9) @ X(10) @ Z(4) @ Z(5))
  + (-1.146777966067637e-07+0j) * (X(3) @ X(6) @ Y(7) @ Z(8) @ Z(9) @ Y(10) @ Z(4) @ Z(5))
  + (1.146777966067637e-07+0j) * (X(3) @ Y(6) @ Y(7) @ Z(8) @ Z(9) @ X(10) @ Z(4) @ Z(5))
  + (1.2070540494600731e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ X(6) @ X(9) @ Y(10))
  + (-1.2070540494600731e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Y(6) @ X(9) @ X(10))
  + (-1.2070540494600731e-08+0j) * (X(3) @ Z(4) @ Z(5) @ X(6) @ Y(9) @ Y(10))
  + (1.2070540494600731e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Y(6) @ Y(9) @ X(10))
  + (-3.0527075409738795e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ X(6) @ X(10) @ Y(11))
  + (-3.0527075409738795e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Y(6) @ Y(10) @ Y(11))
  + (-3.0527075409738795e-07+0j) * (X(3) @ Z(4) @ Z(5) @ X(6) @ X(10) @ X(11))
  + (-3.0527075409738795e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Y(6) @ Y(10) @ X(11))
  + (4.776953729518488e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Y(9))
  + (4.776953729518488e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ X(9))
  + (3.9790052479638824e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (3.9790052479638824e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-1.4481289214215305e-06+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(9))
  + (-1.4481289214215305e-06+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(9))
  + (8.194466821363216e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ Z(10) @ Y(11))
  + (5.367507516707476e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ Z(10) @ Y(11))
  + (-2.82695930465574e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(9) @ Z(10) @ X(11))
  + (-2.82695930465574e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(9) @ Z(10) @ Y(11))
  + (5.367507516707476e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ Z(10) @ X(11))
  + (8.194466821363216e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ Z(10) @ X(11))
  + (-4.9338304715739734e-06+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(11))
  + (-4.9338304715739734e-06+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(11))
  + (0.08426885389982977+0j) * (Z(3) @ Z(8))
  + (1.1741109234098053e-07+0j) * (Y(3) @ Z(4) @ Y(5) @ Z(8))
  + (1.1741109234098053e-07+0j) * (X(3) @ Z(4) @ X(5) @ Z(8))
  + (-9.51016934212325e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(8))
  + (-9.51016934212325e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(8))
  + (2.1140708919878435e-07+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(9))
  + (2.1140708919878435e-07+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(9))
  + (-4.794572673278472e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ Z(10) @ Y(11))
  + (-4.794572673278472e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ Z(10) @ X(11))
  + (-0.00015493599386944468+0j) * (Z(3) @ Y(8) @ Z(9) @ Y(10))
  + (-0.00015493599386944468+0j) * (Z(3) @ X(8) @ Z(9) @ X(10))
  + (-4.3753773000814385e-05+0j) * (Y(3) @ Y(8) @ Z(9) @ Y(10) @ Z(4) @ Y(5))
  + (-4.3753773000814385e-05+0j) * (Y(3) @ X(8) @ Z(9) @ X(10) @ Z(4) @ Y(5))
  + (-4.3753773000814385e-05+0j) * (X(3) @ Y(8) @ Z(9) @ Y(10) @ Z(4) @ X(5))
  + (-4.3753773000814385e-05+0j) * (X(3) @ X(8) @ Z(9) @ X(10) @ Z(4) @ X(5))
  + (-1.6199052551956667e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Y(8) @ Z(9) @ Y(10))
  + (-1.6199052551956667e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ X(8) @ Z(9) @ X(10))
  + (-1.6199052551956667e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Y(8) @ Z(9) @ Y(10))
  + (-1.6199052551956667e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ X(8) @ Z(9) @ X(10))
  + (4.159734484859583e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(9) @ Y(10))
  + (-4.159734484859583e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ X(9) @ X(10))
  + (-4.159734484859583e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Y(9) @ Y(10))
  + (4.159734484859583e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(9) @ X(10))
  + (2.403256387902117e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(10) @ Y(11))
  + (2.403256387902117e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(10) @ Y(11))
  + (2.403256387902117e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(10) @ X(11))
  + (2.403256387902117e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(10) @ X(11))
  + (-6.3483818841888935e-09+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(10) @ Y(11))
  + (-6.3483818841888935e-09+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(10) @ X(11))
  + (4.9382259879352136e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(11))
  + (4.9382259879352136e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(11))
  + (0.19518208290819333+0j) * (Z(3) @ Z(10))
  + (2.5630797543135006e-06+0j) * (Y(3) @ Z(4) @ Y(5) @ Z(10))
  + (2.5630797543135006e-06+0j) * (X(3) @ Z(4) @ X(5) @ Z(10))
  + (-5.239101225671361e-06+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Y(7) @ Z(10))
  + (-5.239101225671361e-06+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ X(7) @ Z(10))
  + (7.34148237583733e-08+0j) * (Y(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(10))
  + (7.34148237583733e-08+0j) * (X(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(10))
  + (0.1627464110766801+0j) * (Z(4) @ Z(6))
  + (-0.01676031747140117+0j) * (Z(4) @ Y(6) @ Z(7) @ Y(8))
  + (-0.01676031747140117+0j) * (Z(4) @ X(6) @ Z(7) @ X(8))
  + (0.005063916188437916+0j) * (Z(4) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.005063916188437916+0j) * (Z(4) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (0.08190946936088947+0j) * (Z(4) @ Z(8))
  + (0.0009605627650653239+0j) * (Z(4) @ Y(8) @ Z(9) @ Y(10))
  + (0.0009605627650653239+0j) * (Z(4) @ X(8) @ Z(9) @ X(10))
  + (0.18721320674328024+0j) * (Z(4) @ Z(10))
  + (0.20347850270504608+0j) * (Z(4) @ Z(5))
  + (1.0065229860882111e-05+0j) * (Y(4) @ Y(6))
  + (1.0065229860882111e-05+0j) * (X(4) @ X(6))
  + (2.9427205352441e-06+0j) * (Y(4) @ Z(6) @ Z(7) @ Y(8))
  + (2.9427205352441e-06+0j) * (X(4) @ Z(6) @ Z(7) @ X(8))
  + (-9.79238832394733e-08+0j) * (Y(4) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (-9.79238832394733e-08+0j) * (X(4) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (1.0065229860882111e-05+0j) * (Z(4) @ Y(5) @ Z(6) @ Y(7))
  + (1.0065229860882111e-05+0j) * (Z(4) @ X(5) @ Z(6) @ X(7))
  + (0.0045037413811850405+0j) * (Y(4) @ X(5) @ X(6) @ Y(7))
  + (-0.0045037413811850405+0j) * (Y(4) @ Y(5) @ X(6) @ X(7))
  + (-0.0045037413811850405+0j) * (X(4) @ X(5) @ Y(6) @ Y(7))
  + (0.0045037413811850405+0j) * (X(4) @ Y(5) @ Y(6) @ X(7))
  + (-0.0009350549728534443+0j) * (Y(4) @ X(5) @ X(7) @ Y(8))
  + (-0.0009350549728534443+0j) * (Y(4) @ Y(5) @ Y(7) @ Y(8))
  + (-0.0009350549728534443+0j) * (X(4) @ X(5) @ X(7) @ X(8))
  + (-0.0009350549728534443+0j) * (X(4) @ Y(5) @ Y(7) @ X(8))
  + (-0.0017276384318794418+0j) * (Y(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0017276384318794418+0j) * (Y(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (-0.0017276384318794418+0j) * (X(4) @ X(5) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (-0.0017276384318794418+0j) * (X(4) @ Y(5) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (2.9427205352441e-06+0j) * (Z(4) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9))
  + (2.9427205352441e-06+0j) * (Z(4) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9))
  + (0.0009350549728534443+0j) * (Y(4) @ X(5) @ X(6) @ Z(7) @ Z(8) @ Y(9))
  + (-0.0009350549728534443+0j) * (Y(4) @ Y(5) @ X(6) @ Z(7) @ Z(8) @ X(9))
  + (-0.0009350549728534443+0j) * (X(4) @ X(5) @ Y(6) @ Z(7) @ Z(8) @ Y(9))
  + (0.0009350549728534443+0j) * (X(4) @ Y(5) @ Y(6) @ Z(7) @ Z(8) @ X(9))
  + (0.002359443295757707+0j) * (Y(4) @ X(5) @ X(8) @ Y(9))
  + (-0.002359443295757707+0j) * (Y(4) @ Y(5) @ X(8) @ X(9))
  + (-0.002359443295757707+0j) * (X(4) @ X(5) @ Y(8) @ Y(9))
  + (0.002359443295757707+0j) * (X(4) @ Y(5) @ Y(8) @ X(9))
  + (-0.0008314013416293416+0j) * (Y(4) @ X(5) @ X(9) @ Y(10))
  + (-0.0008314013416293416+0j) * (Y(4) @ Y(5) @ Y(9) @ Y(10))
  + (-0.0008314013416293416+0j) * (X(4) @ X(5) @ X(9) @ X(10))
  + (-0.0008314013416293416+0j) * (X(4) @ Y(5) @ Y(9) @ X(10))
  + (-9.79238832394733e-08+0j) * (Z(4) @ Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-9.79238832394733e-08+0j) * (Z(4) @ X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.0017276384318794418+0j) * (Y(4) @ X(5) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0017276384318794418+0j) * (Y(4) @ Y(5) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.0017276384318794418+0j) * (X(4) @ X(5) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.0017276384318794418+0j) * (X(4) @ Y(5) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.0008314013416293416+0j) * (Y(4) @ X(5) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (-0.0008314013416293416+0j) * (Y(4) @ Y(5) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.0008314013416293416+0j) * (X(4) @ X(5) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.0008314013416293416+0j) * (X(4) @ Y(5) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (0.007956504525043357+0j) * (Y(4) @ X(5) @ X(10) @ Y(11))
  + (-0.007956504525043357+0j) * (Y(4) @ Y(5) @ X(10) @ X(11))
  + (-0.007956504525043357+0j) * (X(4) @ X(5) @ Y(10) @ Y(11))
  + (0.007956504525043357+0j) * (X(4) @ Y(5) @ Y(10) @ X(11))
  + (1.345241191733723e-06+0j) * (Y(4) @ Z(5) @ Z(7) @ Y(8))
  + (1.345241191733723e-06+0j) * (X(4) @ Z(5) @ Z(7) @ X(8))
  + (3.355720903515011e-07+0j) * (Y(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (3.355720903515011e-07+0j) * (X(4) @ Z(5) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (2.4096038512476376e-06+0j) * (Y(4) @ Z(5) @ Y(6) @ Z(8))
  + (2.4096038512476376e-06+0j) * (X(4) @ Z(5) @ X(6) @ Z(8))
  + (1.415798958631241e-07+0j) * (Y(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ Y(10))
  + (1.3289165458323908e-09+0j) * (Y(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ Y(10))
  + (-1.4025097931729172e-07+0j) * (Y(4) @ Z(5) @ Y(6) @ X(8) @ Z(9) @ X(10))
  + (-1.4025097931729172e-07+0j) * (X(4) @ Z(5) @ X(6) @ Y(8) @ Z(9) @ Y(10))
  + (1.3289165458323908e-09+0j) * (X(4) @ Z(5) @ X(6) @ X(8) @ Z(9) @ X(10))
  + (1.415798958631241e-07+0j) * (X(4) @ Z(5) @ Y(6) @ Y(8) @ Z(9) @ X(10))
  + (9.264146585305686e-06+0j) * (Y(4) @ Z(5) @ Y(6) @ Z(10))
  + (9.264146585305686e-06+0j) * (X(4) @ Z(5) @ X(6) @ Z(10))
  + (0.16725015245786512+0j) * (Z(4) @ Z(7))
  + (7.197839870901928e-06+0j) * (Y(4) @ Z(5) @ Y(6) @ Z(7))
  + (7.197839870901928e-06+0j) * (X(4) @ Z(5) @ X(6) @ Z(7))
  + (3.5072062506140697e-06+0j) * (Y(4) @ Z(5) @ Z(6) @ Y(8))
  + (3.5072062506140697e-06+0j) * (X(4) @ Z(5) @ Z(6) @ X(8))
  + (2.93732413114536e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ Y(10))
  + (2.93732413114536e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(8) @ Z(9) @ X(10))
  + (-0.01769537244425461+0j) * (Z(4) @ Y(7) @ Z(8) @ Y(9))
  + (-0.01769537244425461+0j) * (Z(4) @ X(7) @ Z(8) @ X(9))
  + (-2.161965058880347e-06+0j) * (Y(4) @ Y(7) @ Z(8) @ Y(9) @ Z(5) @ Y(6))
  + (-2.161965058880347e-06+0j) * (Y(4) @ X(7) @ Z(8) @ X(9) @ Z(5) @ Y(6))
  + (-2.161965058880347e-06+0j) * (X(4) @ Y(7) @ Z(8) @ Y(9) @ Z(5) @ X(6))
  + (-2.161965058880347e-06+0j) * (X(4) @ X(7) @ Z(8) @ X(9) @ Z(5) @ X(6))
  + (6.52967070570189e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ X(7) @ X(8) @ Y(9))
  + (-6.52967070570189e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Y(7) @ X(8) @ X(9))
  + (-6.52967070570189e-07+0j) * (X(4) @ Z(5) @ Z(6) @ X(7) @ Y(8) @ Y(9))
  + (6.52967070570189e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Y(7) @ Y(8) @ X(9))
  + (3.530088781190963e-08+0j) * (Y(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ Y(10))
  + (3.530088781190963e-08+0j) * (Y(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ Y(10))
  + (3.530088781190963e-08+0j) * (X(4) @ Z(5) @ Z(6) @ X(7) @ X(9) @ X(10))
  + (3.530088781190963e-08+0j) * (X(4) @ Z(5) @ Z(6) @ Y(7) @ Y(9) @ X(10))
  + (0.003336277756558474+0j) * (Z(4) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.003336277756558474+0j) * (Z(4) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (4.183967723696509e-08+0j) * (Y(4) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(5) @ Y(6))
  + (4.183967723696509e-08+0j) * (Y(4) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(5) @ Y(6))
  + (4.183967723696509e-08+0j) * (X(4) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11) @ Z(5) @ X(6))
  + (4.183967723696509e-08+0j) * (X(4) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11) @ Z(5) @ X(6))
  + (1.062790080512145e-07+0j) * (Y(4) @ X(7) @ X(8) @ Z(9) @ Z(10) @ Y(11) @ Z(5) @ Z(6))
  + (-1.062790080512145e-07+0j) * (Y(4) @ Y(7) @ X(8) @ Z(9) @ Z(10) @ X(11) @ Z(5) @ Z(6))
  + (-1.062790080512145e-07+0j) * (X(4) @ X(7) @ Y(8) @ Z(9) @ Z(10) @ Y(11) @ Z(5) @ Z(6))
  + (1.062790080512145e-07+0j) * (X(4) @ Y(7) @ Y(8) @ Z(9) @ Z(10) @ X(11) @ Z(5) @ Z(6))
  + (-6.44815361213766e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ X(7) @ X(10) @ Y(11))
  + (6.44815361213766e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Y(7) @ X(10) @ X(11))
  + (6.44815361213766e-07+0j) * (X(4) @ Z(5) @ Z(6) @ X(7) @ Y(10) @ Y(11))
  + (-6.44815361213766e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Y(7) @ Y(10) @ X(11))
  + (-3.2308132338402066e-08+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ Y(10))
  + (-3.2308132338402066e-08+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(9) @ X(10))
  + (2.5951241063855575e-06+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(10))
  + (2.5951241063855575e-06+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(10))
  + (0.08426891265664718+0j) * (Z(4) @ Z(9))
  + (1.7566367806774485e-06+0j) * (Y(4) @ Z(5) @ Y(6) @ Z(9))
  + (1.7566367806774485e-06+0j) * (X(4) @ Z(5) @ X(6) @ Z(9))
  + (-5.594207226614856e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(9))
  + (-5.594207226614856e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(9))
  + (-1.831135144941702e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(10))
  + (-1.831135144941702e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(10))
  + (0.00012916142343598222+0j) * (Z(4) @ Y(9) @ Z(10) @ Y(11))
  + (0.00012916142343598222+0j) * (Z(4) @ X(9) @ Z(10) @ X(11))
  + (-3.397197126607724e-08+0j) * (Y(4) @ Y(9) @ Z(10) @ Y(11) @ Z(5) @ Y(6))
  + (-3.397197126607724e-08+0j) * (Y(4) @ X(9) @ Z(10) @ X(11) @ Z(5) @ Y(6))
  + (-3.397197126607724e-08+0j) * (X(4) @ Y(9) @ Z(10) @ Y(11) @ Z(5) @ X(6))
  + (-3.397197126607724e-08+0j) * (X(4) @ X(9) @ Z(10) @ X(11) @ Z(5) @ X(6))
  + (1.5080538215576813e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Y(9) @ Z(10) @ Y(11))
  + (1.5080538215576813e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ X(9) @ Z(10) @ X(11))
  + (1.5080538215576813e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Y(9) @ Z(10) @ Y(11))
  + (1.5080538215576813e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ X(9) @ Z(10) @ X(11))
  + (-1.3564233963335465e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ X(10) @ Y(11))
  + (1.3564233963335465e-07+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ X(10) @ X(11))
  + (1.3564233963335465e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Y(10) @ Y(11))
  + (-1.3564233963335465e-07+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Y(10) @ X(11))
  + (0.19516971126832358+0j) * (Z(4) @ Z(11))
  + (9.908961946519452e-06+0j) * (Y(4) @ Z(5) @ Y(6) @ Z(11))
  + (9.908961946519452e-06+0j) * (X(4) @ Z(5) @ X(6) @ Z(11))
  + (2.7307664460189124e-06+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Y(8) @ Z(11))
  + (2.7307664460189124e-06+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ X(8) @ Z(11))
  + (9.663631478315608e-08+0j) * (Y(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(11))
  + (9.663631478315608e-08+0j) * (X(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(11))
  + (0.1627464110766801+0j) * (Z(5) @ Z(7))
  + (-0.01676031747140117+0j) * (Z(5) @ Y(7) @ Z(8) @ Y(9))
  + (-0.01676031747140117+0j) * (Z(5) @ X(7) @ Z(8) @ X(9))
  + (0.005063916188437916+0j) * (Z(5) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (0.005063916188437916+0j) * (Z(5) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (0.08190946936088947+0j) * (Z(5) @ Z(9))
  + (0.0009605627650653239+0j) * (Z(5) @ Y(9) @ Z(10) @ Y(11))
  + (0.0009605627650653239+0j) * (Z(5) @ X(9) @ Z(10) @ X(11))
  + (0.18721320674328024+0j) * (Z(5) @ Z(11))
  + (0.16725015245786512+0j) * (Z(5) @ Z(6))
  + (7.197839870901928e-06+0j) * (Y(5) @ Y(7))
  + (7.197839870901928e-06+0j) * (X(5) @ X(7))
  + (3.5072062506140697e-06+0j) * (Y(5) @ Z(7) @ Z(8) @ Y(9))
  + (3.5072062506140697e-06+0j) * (X(5) @ Z(7) @ Z(8) @ X(9))
  + (2.93732413114536e-07+0j) * (Y(5) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (2.93732413114536e-07+0j) * (X(5) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (-0.01769537244425461+0j) * (Z(5) @ Y(6) @ Z(7) @ Y(8))
  + (-0.01769537244425461+0j) * (Z(5) @ X(6) @ Z(7) @ X(8))
  + (-2.161965058880347e-06+0j) * (Y(5) @ X(6) @ X(7) @ Y(8))
  + (2.161965058880347e-06+0j) * (Y(5) @ Y(6) @ X(7) @ X(8))
  + (2.161965058880347e-06+0j) * (X(5) @ X(6) @ Y(7) @ Y(8))
  + (-2.161965058880347e-06+0j) * (X(5) @ Y(6) @ Y(7) @ X(8))
  + (-6.52967070570189e-07+0j) * (Y(5) @ X(6) @ X(8) @ Y(9))
  + (-6.52967070570189e-07+0j) * (Y(5) @ Y(6) @ Y(8) @ Y(9))
  + (-6.52967070570189e-07+0j) * (X(5) @ X(6) @ X(8) @ X(9))
  + (-6.52967070570189e-07+0j) * (X(5) @ Y(6) @ Y(8) @ X(9))
  + (3.530088781190963e-08+0j) * (Y(5) @ X(6) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (3.530088781190963e-08+0j) * (Y(5) @ Y(6) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (3.530088781190963e-08+0j) * (X(5) @ X(6) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (3.530088781190963e-08+0j) * (X(5) @ Y(6) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (0.003336277756558474+0j) * (Z(5) @ Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10))
  + (0.003336277756558474+0j) * (Z(5) @ X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10))
  + (4.183967723696509e-08+0j) * (Y(5) @ X(6) @ X(7) @ Z(8) @ Z(9) @ Y(10))
  + (-4.183967723696509e-08+0j) * (Y(5) @ Y(6) @ X(7) @ Z(8) @ Z(9) @ X(10))
  + (-4.183967723696509e-08+0j) * (X(5) @ X(6) @ Y(7) @ Z(8) @ Z(9) @ Y(10))
  + (4.183967723696509e-08+0j) * (X(5) @ Y(6) @ Y(7) @ Z(8) @ Z(9) @ X(10))
  + (1.062790080512145e-07+0j) * (Y(5) @ X(6) @ X(9) @ Y(10))
  + (-1.062790080512145e-07+0j) * (Y(5) @ Y(6) @ X(9) @ X(10))
  + (-1.062790080512145e-07+0j) * (X(5) @ X(6) @ Y(9) @ Y(10))
  + (1.062790080512145e-07+0j) * (X(5) @ Y(6) @ Y(9) @ X(10))
  + (6.44815361213766e-07+0j) * (Y(5) @ X(6) @ X(10) @ Y(11))
  + (6.44815361213766e-07+0j) * (Y(5) @ Y(6) @ Y(10) @ Y(11))
  + (6.44815361213766e-07+0j) * (X(5) @ X(6) @ X(10) @ X(11))
  + (6.44815361213766e-07+0j) * (X(5) @ Y(6) @ Y(10) @ X(11))
  + (1.345241191733723e-06+0j) * (Y(5) @ Z(6) @ Z(8) @ Y(9))
  + (1.345241191733723e-06+0j) * (X(5) @ Z(6) @ Z(8) @ X(9))
  + (3.355720903515011e-07+0j) * (Y(5) @ Z(6) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (3.355720903515011e-07+0j) * (X(5) @ Z(6) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (2.4096038512476376e-06+0j) * (Y(5) @ Z(6) @ Y(7) @ Z(9))
  + (2.4096038512476376e-06+0j) * (X(5) @ Z(6) @ X(7) @ Z(9))
  + (1.415798958631241e-07+0j) * (Y(5) @ Z(6) @ X(7) @ X(9) @ Z(10) @ Y(11))
  + (1.3289165458323908e-09+0j) * (Y(5) @ Z(6) @ Y(7) @ Y(9) @ Z(10) @ Y(11))
  + (-1.4025097931729172e-07+0j) * (Y(5) @ Z(6) @ Y(7) @ X(9) @ Z(10) @ X(11))
  + (-1.4025097931729172e-07+0j) * (X(5) @ Z(6) @ X(7) @ Y(9) @ Z(10) @ Y(11))
  + (1.3289165458323908e-09+0j) * (X(5) @ Z(6) @ X(7) @ X(9) @ Z(10) @ X(11))
  + (1.415798958631241e-07+0j) * (X(5) @ Z(6) @ Y(7) @ Y(9) @ Z(10) @ X(11))
  + (9.264146585305686e-06+0j) * (Y(5) @ Z(6) @ Y(7) @ Z(11))
  + (9.264146585305686e-06+0j) * (X(5) @ Z(6) @ X(7) @ Z(11))
  + (0.08426891265664718+0j) * (Z(5) @ Z(8))
  + (1.7566367806774485e-06+0j) * (Y(5) @ Z(6) @ Y(7) @ Z(8))
  + (1.7566367806774485e-06+0j) * (X(5) @ Z(6) @ X(7) @ Z(8))
  + (-5.594207226614856e-07+0j) * (Y(5) @ Z(6) @ Z(7) @ Y(9))
  + (-5.594207226614856e-07+0j) * (X(5) @ Z(6) @ Z(7) @ X(9))
  + (-1.831135144941702e-07+0j) * (Y(5) @ Z(6) @ Z(7) @ Z(9) @ Z(10) @ Y(11))
  + (-1.831135144941702e-07+0j) * (X(5) @ Z(6) @ Z(7) @ Z(9) @ Z(10) @ X(11))
  + (0.00012916142343598222+0j) * (Z(5) @ Y(8) @ Z(9) @ Y(10))
  + (0.00012916142343598222+0j) * (Z(5) @ X(8) @ Z(9) @ X(10))
  + (-3.397197126607724e-08+0j) * (Y(5) @ Y(8) @ Z(9) @ Y(10) @ Z(6) @ Y(7))
  + (-3.397197126607724e-08+0j) * (Y(5) @ X(8) @ Z(9) @ X(10) @ Z(6) @ Y(7))
  + (-3.397197126607724e-08+0j) * (X(5) @ Y(8) @ Z(9) @ Y(10) @ Z(6) @ X(7))
  + (-3.397197126607724e-08+0j) * (X(5) @ X(8) @ Z(9) @ X(10) @ Z(6) @ X(7))
  + (1.5080538215576813e-07+0j) * (Y(5) @ Z(6) @ Z(7) @ X(8) @ X(9) @ Y(10))
  + (-1.5080538215576813e-07+0j) * (Y(5) @ Z(6) @ Z(7) @ Y(8) @ X(9) @ X(10))
  + (-1.5080538215576813e-07+0j) * (X(5) @ Z(6) @ Z(7) @ X(8) @ Y(9) @ Y(10))
  + (1.5080538215576813e-07+0j) * (X(5) @ Z(6) @ Z(7) @ Y(8) @ Y(9) @ X(10))
  + (1.3564233963335465e-07+0j) * (Y(5) @ Z(6) @ Z(7) @ X(8) @ X(10) @ Y(11))
  + (1.3564233963335465e-07+0j) * (Y(5) @ Z(6) @ Z(7) @ Y(8) @ Y(10) @ Y(11))
  + (1.3564233963335465e-07+0j) * (X(5) @ Z(6) @ Z(7) @ X(8) @ X(10) @ X(11))
  + (1.3564233963335465e-07+0j) * (X(5) @ Z(6) @ Z(7) @ Y(8) @ Y(10) @ X(11))
  + (-3.2308132338402066e-08+0j) * (Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(10) @ Y(11))
  + (-3.2308132338402066e-08+0j) * (X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(10) @ X(11))
  + (2.5951241063855575e-06+0j) * (Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(11))
  + (2.5951241063855575e-06+0j) * (X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(11))
  + (0.19516971126832358+0j) * (Z(5) @ Z(10))
  + (9.908961946519452e-06+0j) * (Y(5) @ Z(6) @ Y(7) @ Z(10))
  + (9.908961946519452e-06+0j) * (X(5) @ Z(6) @ X(7) @ Z(10))
  + (2.7307664460189124e-06+0j) * (Y(5) @ Z(6) @ Z(7) @ Z(8) @ Y(9) @ Z(10))
  + (2.7307664460189124e-06+0j) * (X(5) @ Z(6) @ Z(7) @ Z(8) @ X(9) @ Z(10))
  + (9.66363147831561e-08+0j) * (Y(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(11))
  + (9.66363147831561e-08+0j) * (X(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(11))
  + (0.0721527319017861+0j) * (Z(6) @ Z(8))
  + (-5.698462077315415e-06+0j) * (Z(6) @ Y(8) @ Z(9) @ Y(10))
  + (-5.698462077315415e-06+0j) * (Z(6) @ X(8) @ Z(9) @ X(10))
  + (0.1590847014653794+0j) * (Z(6) @ Z(10))
  + (0.1584148855684904+0j) * (Z(6) @ Z(7))
  + (-0.011468335461147146+0j) * (Y(6) @ Y(8))
  + (-0.011468335461147146+0j) * (X(6) @ X(8))
  + (-2.6654496530498486e-05+0j) * (Y(6) @ Z(8) @ Z(9) @ Y(10))
  + (-2.6654496530498486e-05+0j) * (X(6) @ Z(8) @ Z(9) @ X(10))
  + (-0.011468335461147146+0j) * (Z(6) @ Y(7) @ Z(8) @ Y(9))
  + (-0.011468335461147146+0j) * (Z(6) @ X(7) @ Z(8) @ X(9))
  + (0.007529717020741537+0j) * (Y(6) @ X(7) @ X(8) @ Y(9))
  + (-0.007529717020741537+0j) * (Y(6) @ Y(7) @ X(8) @ X(9))
  + (-0.007529717020741537+0j) * (X(6) @ X(7) @ Y(8) @ Y(9))
  + (0.007529717020741537+0j) * (X(6) @ Y(7) @ Y(8) @ X(9))
  + (-9.063655745444726e-06+0j) * (Y(6) @ X(7) @ X(9) @ Y(10))
  + (-9.063655745444726e-06+0j) * (Y(6) @ Y(7) @ Y(9) @ Y(10))
  + (-9.063655745444726e-06+0j) * (X(6) @ X(7) @ X(9) @ X(10))
  + (-9.063655745444726e-06+0j) * (X(6) @ Y(7) @ Y(9) @ X(10))
  + (-2.6654496530498486e-05+0j) * (Z(6) @ Y(7) @ Z(8) @ Z(9) @ Z(10) @ Y(11))
  + (-2.6654496530498486e-05+0j) * (Z(6) @ X(7) @ Z(8) @ Z(9) @ Z(10) @ X(11))
  + (9.063655745444728e-06+0j) * (Y(6) @ X(7) @ X(8) @ Z(9) @ Z(10) @ Y(11))
  + (-9.063655745444728e-06+0j) * (Y(6) @ Y(7) @ X(8) @ Z(9) @ Z(10) @ X(11))
  + (-9.063655745444728e-06+0j) * (X(6) @ X(7) @ Y(8) @ Z(9) @ Z(10) @ Y(11))
  + (9.063655745444728e-06+0j) * (X(6) @ Y(7) @ Y(8) @ Z(9) @ Z(10) @ X(11))
  + (0.00714040310082522+0j) * (Y(6) @ X(7) @ X(10) @ Y(11))
  + (-0.00714040310082522+0j) * (Y(6) @ Y(7) @ X(10) @ X(11))
  + (-0.00714040310082522+0j) * (X(6) @ X(7) @ Y(10) @ Y(11))
  + (0.00714040310082522+0j) * (X(6) @ Y(7) @ Y(10) @ X(11))
  + (-9.008341768796045e-06+0j) * (Y(6) @ Z(7) @ Z(9) @ Y(10))
  + (-9.008341768796045e-06+0j) * (X(6) @ Z(7) @ Z(9) @ X(10))
  + (-0.017034416065387022+0j) * (Y(6) @ Z(7) @ Y(8) @ Z(10))
  + (-0.017034416065387022+0j) * (X(6) @ Z(7) @ X(8) @ Z(10))
  + (0.07968244892252765+0j) * (Z(6) @ Z(9))
  + (-0.005639889929667717+0j) * (Y(6) @ Z(7) @ Y(8) @ Z(9))
  + (-0.005639889929667717+0j) * (X(6) @ Z(7) @ X(8) @ Z(9))
  + (-7.696847090431957e-06+0j) * (Y(6) @ Z(7) @ Z(8) @ Y(10))
  + (-7.696847090431957e-06+0j) * (X(6) @ Z(7) @ Z(8) @ X(10))
  + (-1.4762117822760144e-05+0j) * (Z(6) @ Y(9) @ Z(10) @ Y(11))
  + (-1.4762117822760144e-05+0j) * (Z(6) @ X(9) @ Z(10) @ X(11))
  + (-1.3114946783640866e-06+0j) * (Y(6) @ Y(9) @ Z(10) @ Y(11) @ Z(7) @ Y(8))
  + (-1.3114946783640866e-06+0j) * (Y(6) @ X(9) @ Z(10) @ X(11) @ Z(7) @ Y(8))
  + (-1.3114946783640866e-06+0j) * (X(6) @ Y(9) @ Z(10) @ Y(11) @ Z(7) @ X(8))
  + (-1.3114946783640866e-06+0j) * (X(6) @ X(9) @ Z(10) @ X(11) @ Z(7) @ X(8))
  + (0.0009659183575637378+0j) * (Y(6) @ Z(7) @ Z(8) @ X(9) @ X(10) @ Y(11))
  + (-0.0009659183575637378+0j) * (Y(6) @ Z(7) @ Z(8) @ Y(9) @ X(10) @ X(11))
  + (-0.0009659183575637378+0j) * (X(6) @ Z(7) @ Z(8) @ X(9) @ Y(10) @ Y(11))
  + (0.0009659183575637378+0j) * (X(6) @ Z(7) @ Z(8) @ Y(9) @ Y(10) @ X(11))
  + (0.1662251045662046+0j) * (Z(6) @ Z(11))
  + (-0.01800033442295076+0j) * (Y(6) @ Z(7) @ Y(8) @ Z(11))
  + (-0.01800033442295076+0j) * (X(6) @ Z(7) @ X(8) @ Z(11))
  + (-4.108422023553416e-05+0j) * (Y(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10) @ Z(11))
  + (-4.108422023553416e-05+0j) * (X(6) @ Z(7) @ Z(8) @ Z(9) @ X(10) @ Z(11))
  + (0.0721527319017861+0j) * (Z(7) @ Z(9))
  + (-5.698462077315415e-06+0j) * (Z(7) @ Y(9) @ Z(10) @ Y(11))
  + (-5.698462077315415e-06+0j) * (Z(7) @ X(9) @ Z(10) @ X(11))
  + (0.1590847014653794+0j) * (Z(7) @ Z(11))
  + (0.07968244892252765+0j) * (Z(7) @ Z(8))
  + (-0.005639889929667718+0j) * (Y(7) @ Y(9))
  + (-0.005639889929667718+0j) * (X(7) @ X(9))
  + (-7.696847090431957e-06+0j) * (Y(7) @ Z(9) @ Z(10) @ Y(11))
  + (-7.696847090431957e-06+0j) * (X(7) @ Z(9) @ Z(10) @ X(11))
  + (-1.4762117822760144e-05+0j) * (Z(7) @ Y(8) @ Z(9) @ Y(10))
  + (-1.4762117822760144e-05+0j) * (Z(7) @ X(8) @ Z(9) @ X(10))
  + (-1.3114946783640866e-06+0j) * (Y(7) @ X(8) @ X(9) @ Y(10))
  + (1.3114946783640866e-06+0j) * (Y(7) @ Y(8) @ X(9) @ X(10))
  + (1.3114946783640866e-06+0j) * (X(7) @ X(8) @ Y(9) @ Y(10))
  + (-1.3114946783640866e-06+0j) * (X(7) @ Y(8) @ Y(9) @ X(10))
  + (-0.0009659183575637378+0j) * (Y(7) @ X(8) @ X(10) @ Y(11))
  + (-0.0009659183575637378+0j) * (Y(7) @ Y(8) @ Y(10) @ Y(11))
  + (-0.0009659183575637378+0j) * (X(7) @ X(8) @ X(10) @ X(11))
  + (-0.0009659183575637378+0j) * (X(7) @ Y(8) @ Y(10) @ X(11))
  + (-9.008341768796045e-06+0j) * (Y(7) @ Z(8) @ Z(10) @ Y(11))
  + (-9.008341768796045e-06+0j) * (X(7) @ Z(8) @ Z(10) @ X(11))
  + (-0.017034416065387022+0j) * (Y(7) @ Z(8) @ Y(9) @ Z(11))
  + (-0.017034416065387022+0j) * (X(7) @ Z(8) @ X(9) @ Z(11))
  + (0.1662251045662046+0j) * (Z(7) @ Z(10))
  + (-0.01800033442295076+0j) * (Y(7) @ Z(8) @ Y(9) @ Z(10))
  + (-0.01800033442295076+0j) * (X(7) @ Z(8) @ X(9) @ Z(10))
  + (-4.108422023553416e-05+0j) * (Y(7) @ Z(8) @ Z(9) @ Y(11))
  + (-4.108422023553416e-05+0j) * (X(7) @ Z(8) @ Z(9) @ X(11))
  + (0.08283300015011284+0j) * (Z(8) @ Z(10))
  + (0.0684544019284133+0j) * (Z(8) @ Z(9))
  + (-1.9028665529733982e-06+0j) * (Y(8) @ Y(10))
  + (-1.9028665529733982e-06+0j) * (X(8) @ X(10))
  + (-1.9028665529733982e-06+0j) * (Z(8) @ Y(9) @ Z(10) @ Y(11))
  + (-1.9028665529733982e-06+0j) * (Z(8) @ X(9) @ Z(10) @ X(11))
  + (0.0012632943121946396+0j) * (Y(8) @ X(9) @ X(10) @ Y(11))
  + (-0.0012632943121946396+0j) * (Y(8) @ Y(9) @ X(10) @ X(11))
  + (-0.0012632943121946396+0j) * (X(8) @ X(9) @ Y(10) @ Y(11))
  + (0.0012632943121946396+0j) * (X(8) @ Y(9) @ Y(10) @ X(11))
  + (0.08409629446230749+0j) * (Z(8) @ Z(11))
  + (-1.1624782071042866e-05+0j) * (Y(8) @ Z(9) @ Y(10) @ Z(11))
  + (-1.1624782071042866e-05+0j) * (X(8) @ Z(9) @ X(10) @ Z(11))
  + (0.08283300015011284+0j) * (Z(9) @ Z(11))
  + (0.08409629446230749+0j) * (Z(9) @ Z(10))
  + (-1.1624782071042866e-05+0j) * (Y(9) @ Y(11))
  + (-1.1624782071042866e-05+0j) * (X(9) @ X(11))
  + (0.21938866992968142+0j) * (Z(10) @ Z(11))
))

In [17]:
2**11 + 2**10 + 2**9 + 2**8 + 2**7 + 2**6 

4032

In [8]:
import pennylane as qml
electrons = 6
qubits = 12
hf_state = qchem.hf_state(electrons, qubits)

dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def e():
    qml.BasisStatePreparation(hf_state, wires=range(qubits))
    return qml.expval(H)

energy = e()
print(f"Energy: {energy:.6f} Ha")

Energy: -1654.923114 Ha


## GCIM - FeS

In [4]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
operator_check = []  # To store the highest gradient excitation operators
theta = np.pi/4
print('Theta is', theta)

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity




electrons = 6
orbitals = 12 #Spin orbitals
qubits = 12
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[4032] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
  


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

    
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('The operator pool length is', len(operator_pool))
    states = [hf_state]
    max_operator = None

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        #max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        counts1 = measure(k)  #Measure the current state
        print('The measurement of new state is', counts1)
        print(f'The operator check for this iteration {j} is', operator_check)
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already used
                #print('The current excitation operator is', i) #Current excitation operator - fermionic one
                #print('The value of  operator check is', operator_check)  #Max excitation operator
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)   
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))  # Get the Hamiltonian matrix
        #print('Shape of Hamiltonian matrix is', Ham_matrix.shape)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                
                #print('The value of i and j is', i, j)
                left_op = states[i].T.conj()
                right_op = states[j]
                M[i,j] = left_op.dot(Ham_matrix.dot(right_op)).real
                #print(f'The value of i and j', i, j, 'in M', M[i,j])
                S[i,j] = left_op.dot(right_op).real
                #print(f'The value of i and j', i, j, 'is', S[i,j])
                #S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                #print(f'The value of i and j', i, j, 'is', S[i,j])
                #M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy) 
        
    return ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state


Theta is 0.7853981633974483


In [5]:
ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state = adaptvqe(adapt_it = 4, e_th=1e-12)

HF state is [6.123234e-17+0.j 0.000000e+00+0.j 0.000000e+00+0.j ... 0.000000e+00+0.j
 0.000000e+00+0.j 0.000000e+00+0.j]
HF state is -1654.9231139836268
The operator pool length is 117
The adapt iteration now is 0
The measurement of new state is {'111111000000': 1000}
The operator check for this iteration 0 is []
The highest operator value is 0.27388293952408344 for operator a⁺(0) a⁺(1) a(6) a(7)
Highest gradient excitation is [0, 1, 6, 7]
ash_excitation is [[0, 1, 6, 7]]
The length of ash_excitation before generating matrix is 1
 0: ─╭|Ψ⟩─╭G²(0.79)─┤  State
 1: ─├|Ψ⟩─├G²(0.79)─┤  State
 2: ─├|Ψ⟩─│─────────┤  State
 3: ─├|Ψ⟩─│─────────┤  State
 4: ─├|Ψ⟩─│─────────┤  State
 5: ─├|Ψ⟩─│─────────┤  State
 6: ─├|Ψ⟩─├G²(0.79)─┤  State
 7: ─├|Ψ⟩─╰G²(0.79)─┤  State
 8: ─├|Ψ⟩───────────┤  State
 9: ─├|Ψ⟩───────────┤  State
10: ─├|Ψ⟩───────────┤  State
11: ─╰|Ψ⟩───────────┤  State
State count after Generating function is {'001111110000': 132, '111111000000': 868}
After adding new state, the leng